---
title: "StanCon 2018 Helsinki Intro Workshop: Pest Control Example"
author: "Abdullah Mahmood"
date: today
format:
    html:
        toc: true
        toc-depth: 3
        code-fold: true
        html-math-method:
            method: mathjax
            url: "https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"
editor: source
jupyter: main
---


### Setup


In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import cmdstanpy
from cmdstanpy import CmdStanModel

from matplotlib import style 
style.use('../../../PlottingStyle.mplstyle')

In [ ]:
def StanModel(stan_file: str, stan_code: str) -> CmdStanModel:
    """Load or compile a Stan model"""
    stan_src = f"{stan_file}.stan"

    if not os.path.isfile(stan_file):  
        open(stan_src, 'w').write(stan_code)  # Write Stan code if needed
        return CmdStanModel(stan_file=stan_src, cpp_options={'STAN_THREADS': 'true', 'parallel_chains': 4})
    
    return CmdStanModel(stan_file=stan_src, exe_file=stan_file)

## The Problem

### Background

Imagine that you are a statistician or data scientist working as an independent contractor. One of your clients is a company that owns many residential buildings throughout New York City. The property manager explains that they are concerned about the number of cockroach complaints that they receive from their buildings. Previously the company has offered monthly visits from a pest inspector as a solution to this problem. While this is the default solution of many property managers in NYC, the tenants are rarely home when the inspector visits, and so the manager reasons that this is a relatively expensive solution that is currently not very effective.

One alternative to this problem is to deploy long term bait stations. In this alternative, child and pet safe bait stations are installed throughout the apartment building. Cockroaches obtain quick acting poison from these stations and distribute it throughout the colony. The manufacturer of these bait stations provides some indication of the space-to-bait efficacy, but the manager suspects that this guidance was not calculated with NYC roaches in mind. NYC roaches, the manager rationalizes, have more hustle than traditional roaches; and NYC buildings are built differently than other common residential buildings in the US. This is particularly important as the unit cost for each bait station per year is quite high.

### The Goal

The manager wishes to employ your services to help them to find the optimal number of roach bait stations they should place in each of their buildings in order to minimize the number of cockroach complaints while also keeping expenditure on pest control affordable.

A subset of the company's buildings have been randomly selected for an experiment:

-   At the beginning of each month, a pest inspector randomly places a number of bait stations throughout the building, without knowledge of the current cockroach levels in the building
-   At the end of the month, the manager records the total number of cockroach complaints in that building.
-   The manager would like to determine the optimal number of traps ($\textrm{traps}$) that balances the lost revenue ($R$) that complaints ($\textrm{complaints}$) generate with the all-in cost of maintaining the traps ($\textrm{TC}$).

Fortunately, Bayesian data analysis provides a coherent framework for us to tackle this problem.

Formally, we are interested in finding:

$$
\arg\max_{\textrm{traps} \in \mathbb{N}} \mathbb{E}_{\text{complaints}}[R(\textrm{complaints}(\textrm{traps})) - \textrm{TC}(\textrm{traps})]
$$

The property manager would also, if possible, like to learn how these results generalize to buildings they haven't treated so they can understand the potential costs of pest control at buildings they are acquiring as well as for the rest of their building portfolio.

As the property manager has complete control over the number of traps set, the random variable contributing to this expectation is the number of complaints given the number of traps. We will model the number of complaints as a function of the number of traps.

## The Data

The data provided to us is in a file called `pest_data.csv`. Let's load the data and see what the structure is:


In [ ]:
pest_data = pd.read_csv('data/pest_data.csv', sep=',', header=0, parse_dates=['date'])
pest_data.head()

We have access to the following fields:

-   `complaints`: Number of complaints per building per month
-   `building_id`: The unique building identifier
-   `traps`: The number of traps used per month per building
-   `date`: The date at which the number of complaints are recorded
-   `live_in_super`: An indicator for whether the building as a live-in super
-   `age_of_building`: The age of the building
-   `total_sq_foot`: The total square footage of the building
-   `average_tenant_age`: The average age of the tenants per building
-   `monthly_average_rent`: The average monthly rent per building
-   `floors`: The number of floors per building

First, let's see how many buildings we have data for:


In [ ]:
N_buildings = len(pest_data.building_id.unique())
N_buildings

And make some plots of the raw data:


In [ ]:
plt.clf()
sns.histplot(pest_data.complaints, discrete=True)
plt.show()
plt.close()

In [ ]:
plt.clf()
sns.stripplot(data=pest_data, x='traps', y='complaints', hue='live_in_super', jitter=True)
plt.show()
plt.close()

In [ ]:
#| warning: false
import matplotlib.dates as mdates
plt.clf()

g = sns.FacetGrid(data=pest_data, col='building_id', hue='live_in_super', col_wrap=2, height=2, sharex=False)
g.map_dataframe(sns.lineplot, 'date', 'traps', linestyle='dashed', color='k', label='Number of Traps')
g.map_dataframe(sns.lineplot, x='date', y='complaints', marker='o', label='Number of Complaints')

for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))  # Show only abbreviated month names (e.g., Jan, Feb)
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Set ticks at the start of each month

line_labels = [
    plt.Line2D([0], [0], color='k', linestyle='dashed', label="Number of Traps"),
    plt.Line2D([0], [0], marker='o', color='k', label="Number of Complaints", linestyle='solid')
]
g.add_legend(handles=line_labels, title='Legend', loc='upper right')   

plt.show()
plt.close()

The first question we'll look at is just whether the number of complaints per building per month is associated with the number of bait stations per building per month, ignoring the temporal and across-building variation (we'll come back to those sources of variation later in the document). That requires only two variables, $\textrm{complaints}$ and $\textrm{traps}$. How should we model the number of complaints?

## Bayesian Workflow

-   Exploratory data analysis
-   Prior predictive checking
-   Model fitting and algorithm diagnostics
-   Posterior predictive checking
-   Model comparison (e.g., via cross-validation)

## Modeling count data: Poisson distribution

We already know some rudimentary information about what we should expect. The number of complaints over a month should be either zero or an integer. The property manager tells us that it is possible but unlikely that number of complaints in a given month is zero. Occasionally there are a very large number of complaints in a single month. A common way of modeling this sort of skewed, single bounded count data is as a Poisson random variable. One concern about modeling the outcome variable as Poisson is that the data may be over-dispersed, but we'll start with the Poisson model and then check whether over-dispersion is a problem by comparing our model's predictions to the data.

### Model

Given that we have chosen a Poisson regression, we define the likelihood to be the Poisson probability mass function over the number bait stations placed in the building, denoted below as `traps`. This model assumes that the mean and variance of the outcome variable `complaints` (number of complaints) is the same. We'll investigate whether this is a good assumption after we fit the model.

For building $b = 1,\dots,10$ at time (month) $t = 1,\dots,12$, we have:

$$
\begin{align*}
\textrm{complaints}_{b,t} & \sim \textrm{Poisson}(\lambda_{b,t}) \\
\lambda_{b,t} & = \exp{(\eta_{b,t})} \\
\eta_{b,t} &= \alpha + \beta \, \textrm{traps}_{b,t}
\end{align*}
$$

Let's encode this probability model in a Stan program.

### Writing our first Stan model


In [ ]:
#| output: false
simple_poisson_regression = '''
functions {
  /*
  * Alternative to poisson_log_rng() that 
  * avoids potential numerical problems during warmup
  */
  int poisson_log_safe_rng(real eta) {
    real pois_rate = exp(eta);
    if (pois_rate >= exp(20.79))
      return -9;
    return poisson_rng(pois_rate);
  }
}
data {
    int<lower=1> N;
    array[N] int<lower=0> complaints;
    vector<lower=0>[N] traps;
}
parameters {
    real alpha;
    real beta;
}
model {
  // weakly informative priors:
  // we expect negative slope on traps and a positive intercept,
  // but we will allow ourselves to be wrong
  beta ~ normal(-0.25, 1);
  alpha ~ normal(log(4), 1);
  
  // poisson_log(eta) is more efficient and stable alternative to poisson(exp(eta))
  complaints ~ poisson_log(alpha + beta * traps);
} 
generated quantities {
    array[N] int y_rep;

    for (n in 1:N) {
        y_rep[n] = poisson_log_safe_rng(alpha + beta * traps[n]);
    }
}
'''

simple_poisson_regression_model = StanModel('../stan_models/simple_poisson_regression', simple_poisson_regression)

### Making sure our code is right

However, before we fit the model to real data, we should check that our model works well with simulated data. We'll simulate data according to the model and then check that we can sufficiently recover the parameter values used in the simulation.


In [ ]:
simple_poisson_regression_dgp_model = '''
data {
    int<lower=1> N;
    real<lower=0> mean_traps;
}
model {
} 
generated quantities {
    array[N] int traps;
    array[N] int complaints;
    real alpha = normal_rng(log(4), .1);
    real beta = normal_rng(-0.25, .1);

    for (n in 1:N)  {
        traps[n] = poisson_rng(mean_traps);
        complaints[n] = poisson_log_rng(alpha + beta * traps[n]);
    }
}
'''

Here we will do the *compilation* and *fitting* in two stages to demonstrate what is happening under the hood.

First we will compile the Stan program (`simple_poisson_regression_dgp.stan`) that will generate the fake data.


In [ ]:
#| output: false
simple_poisson_regression_dgp = StanModel('../stan_models/simple_poisson_regression_dgp', simple_poisson_regression_dgp_model)

Now we can simulate the data by calling the `sample()` method on the `CmdStanModel` object instantiated above. The method requires that we pass input data in the form of a dictionary. The names must match the names used in the `data` block of the Stan program.


In [ ]:
#| output: false
fitted_model_dgp = simple_poisson_regression_dgp.sample(
    data={'N': len(pest_data.traps), 
          'mean_traps': pest_data.traps.mean()},
    chains=1,
    iter_sampling=1,
    show_progress=False
)

samps_dgp = az.from_cmdstanpy(fitted_model_dgp)

### Fit the model to the fake data

In order to pass the fake data to our Stan program, we need to arrange the data into a dictionary. The keys must match the names used in the `data` block of the Stan program.


In [ ]:
stan_dat_fake = {
    'N': len(pest_data.traps),
    'traps': samps_dgp.posterior.traps.sel(chain=0, draw=0).to_numpy().astype(np.int16),
    'complaints': samps_dgp.posterior.complaints.sel(chain=0, draw=0).to_numpy().astype(np.int16)
}

Now that we have the simulated data we fit the model to see if we can recover the `alpha` and `beta` parameters used in the simulation.


In [ ]:
#| output: false
fit_model_P = simple_poisson_regression_model.sample(data=stan_dat_fake, show_progress=False)
posterior_alpha_beta = az.extract(az.from_cmdstanpy(fit_model_P), 'posterior', var_names=['alpha', 'beta']).to_dataframe()[['alpha', 'beta']]

### Assess parameter recovery


In [ ]:
true_alpha = samps_dgp.posterior.alpha.sel(chain=0, draw=0).to_numpy()
true_beta = samps_dgp.posterior.beta.sel(chain=0, draw=0).to_numpy()

plt.clf()
_, ax = plt.subplots(1,2, figsize=(8,4))
sns.histplot(posterior_alpha_beta, x='alpha', bins=30, ax=ax[0])
ax[0].axvline(true_alpha, color='r', linewidth=2)
sns.histplot(posterior_alpha_beta, x='beta', bins=30, ax=ax[1])
ax[1].axvline(true_beta, color='r', linewidth=2)
plt.show()
plt.close()

We don't do a great job recovering the parameters here simply because we're simulating so few observations that the posterior uncertainty remains rather large, but it looks at least *plausible* ($\alpha$ and $\beta$ are contained within the histograms). If we did the simulation with many more observations the parameters would be estimated much more precisely.

We should also check if the `y_rep` datasets (in-sample predictions) that we coded in the `generated quantities` block are similar to the `y` (complaints) values we conditioned on when fitting the model.

Here is a plot of the density estimate of the observed data compared to 200 of the `y_rep` datasets:


In [ ]:
y_rep = az.from_cmdstanpy(fit_model_P).posterior.y_rep

plt.clf()
for i in range(200):
    sns.kdeplot(y_rep.sel(chain=slice(0,4), draw=i).to_dataframe(), x='y_rep', color='k', alpha=0.2)
sns.kdeplot(stan_dat_fake['complaints'], color='r', linewidth=2)
plt.xlim(0,12)
plt.show()
plt.close()

In the plot above we have the kernel density estimate of the observed data ($y$, thicker curve) and 200 simulated data sets ($y_{rep}$, thin curves) from the posterior predictive distribution. If the model fits the data well, as it does here, there is little difference between the observed dataset and the simulated datasets.

Another plot we can make for count data is a rootogram. This is a plot of the expected counts (continuous line) vs the observed counts (blue histogram). We can see the model fits well because the observed histogram matches the expected counts relatively well.


In [ ]:
plt.clf()
fig, ax = plt.subplots()
vals, counts = np.unique(y_rep.to_dataframe()['y_rep'].to_numpy(), return_counts=True)
hist_range = (min(vals), max(vals))
az.plot_dist(stan_dat_fake['complaints'], kind='hist', ax=ax, 
             hist_kwargs={'bins': np.arange(hist_range[0], hist_range[1] + 1, 1), 'range': hist_range})

secax = ax.twinx()
secax.plot(vals, counts, color='red', linestyle='dashed', marker='o')

ax.set_xlim(hist_range[0]-0.5, hist_range[1]+0.5)

secax.set_yticklabels([])
secax.yaxis.set_ticks([])
plt.show()
plt.close()

### Fit with real data

To fit the model to the actual observed data we'll first create a dictionary to pass to Stan using the variables in the `pest_data` data frame:


In [ ]:
stan_dat_simple = {
    'N': len(pest_data.traps),
    'complaints': pest_data.complaints,
    'traps': pest_data.traps
}

In [ ]:
#| output: false
fit_P_real_data = simple_poisson_regression_model.sample(data=stan_dat_simple, show_progress=False)
az_fit_P_real_data = az.from_cmdstanpy(fit_P_real_data, posterior_predictive='y_rep')

In [ ]:
summary = fit_P_real_data.summary(percentiles=(2.5, 25, 50, 75, 97.5))
summary[~summary.index.str.contains('y_rep')]

We can also plot the posterior distributions:


In [ ]:
import matplotlib.ticker as mticker
plt.clf()
_, ax = plt.subplots(1,2, figsize=(8,4))
az.plot_dist(az_fit_P_real_data.posterior.alpha, kind='hist', ax=ax[0])
az.plot_dist(az_fit_P_real_data.posterior.beta, kind='hist', ax=ax[1])
for a in ax:
    a.xaxis.set_major_locator(mticker.MaxNLocator(nbins=5))
plt.show()
plt.close()

As we expected, it appears the number of bait stations set in a building is associated with the number of complaints about cockroaches that were made in the following month. However, we still need to consider how well the model fits.

### Posterior predictive checking


In [ ]:
y_rep = az_fit_P_real_data.posterior_predictive.y_rep

plt.clf()
for i in range(200):
    sns.kdeplot(y_rep.sel(chain=slice(0), draw=i).to_dataframe(), x='y_rep', color='k', alpha=0.2)
sns.kdeplot(stan_dat_simple['complaints'], color='r', linewidth=2)
plt.xlim(0,25)
plt.show()
plt.close()

As opposed to when we fit the model to simulated data above, here the simulated datasets is not as dispersed as the observed data and don't seem to capture the rate of zeros in the observed data. The Poisson model may not be sufficient for this data.

Let's explore this further by looking directly at the proportion of zeros in the real data and predicted data.


In [ ]:
plt.clf()
_, ax = plt.subplots()
sns.histplot(np.mean(y_rep == 0, axis=2).to_numpy().flatten(), bins=20, ax=ax)
ax.axvline(np.mean(stan_dat_simple['complaints'] == 0), linewidth=2, color='r')
plt.show()
plt.close()

The plot above shows the observed proportion of zeros (thick vertical line) and a histogram of the proportion of zeros in each of the simulated data sets. It is clear that the model does not capture this feature of the data well at all.

This next plot is a plot of the standardized residuals of the observed vs predicted number of complaints.


In [ ]:
mean_y_rep = y_rep.mean(dim=['chain', 'draw'])
std_resid = (stan_dat_simple['complaints'] - mean_y_rep) / np.sqrt(mean_y_rep)

plt.clf()
plt.plot(mean_y_rep, std_resid, 'o', alpha=0.5)
plt.axhline(2, linestyle='--')
plt.axhline(-2, linestyle='--')
plt.xlabel('mean_y_rep')
plt.ylabel('std_resid')
plt.show()
plt.close()

As you can see here, it looks as though we have more positive residuals than negative, which indicates that the model tends to underestimate the number of complaints that will be received.

Below another useful plot to compare the observed vs expected number of complaints. This is a plot of the expected counts (continuous line) vs the observed counts (black histogram):


In [ ]:
plt.clf()
fig, ax = plt.subplots()
vals, counts = np.unique(y_rep, return_counts=True)
hist_range = (min(vals), max(vals))
az.plot_dist(stan_dat_simple['complaints'], kind='hist', ax=ax, 
             hist_kwargs={'bins': np.arange(hist_range[0], hist_range[1] + 1, 1), 'range': hist_range})

secax = ax.twinx()
secax.plot(vals, counts, color='red', linestyle='dashed', marker='o')

ax.set_xlim(hist_range[0]-0.5, hist_range[1]+0.5)

secax.set_yticklabels([])
secax.yaxis.set_ticks([])
plt.show()
plt.close()

If the model was fitting well these would be relatively similar, however in this figure we can see the number of complaints is underestimated if there are few complaints, over-estimated for medium numbers of complaints, and underestimated if there are a large number of complaints.

We can also view how the predicted number of complaints varies with the number of traps. From this we can see that the model doesn't seem to fully capture the data.


In [ ]:
from collections import defaultdict

plt.clf()
_, ax = plt.subplots()

ax.plot(stan_dat_simple['traps'], stan_dat_simple['complaints'], 'o', fillstyle='none')

grouped_indices = defaultdict(list)
for idx, trap in enumerate(stan_dat_simple['traps']):
    grouped_indices[trap].append(idx)

grouped_array = {trap: y_rep[:, :, idx_list].to_numpy().flatten() for trap, idx_list in grouped_indices.items()}

for traps_x, complaints_y in grouped_array.items():
    inner_prob = np.array([np.quantile(complaints_y, 0.75), np.quantile(complaints_y, 0.25)])  # az.hdi(complaints_y, hdi_prob=0.5)
    outer_prob = np.array([np.quantile(complaints_y, 0.95), np.quantile(complaints_y, 0.05)])  # az.hdi(complaints_y, hdi_prob=0.9)
    ax.plot(np.full(inner_prob.shape, traps_x), inner_prob, 'k', linewidth=2)
    ax.plot(np.full(outer_prob.shape, traps_x), outer_prob, 'k', linewidth=0.5)
    ax.plot(traps_x, np.mean(complaints_y), 'ko')

plt.show()
plt.close()

Specifically, the model doesn't capture the tails of the observed data very well.

## Expanding the model: multiple predictors

Modeling the relationship between complaints and bait stations is the simplest model. We can expand the model, however, in a few ways that will be beneficial for our client. Moreover, the manager has told us that they expect there are a number of other reasons that one building might have more roach complaints than another.

### Interpretability

Currently, our model's mean parameter is a rate of complaints per 30 days, but we're modeling a process that occurs over an area as well as over time. We have the square footage of each building, so if we add that information into the model, we can interpret our parameters as a rate of complaints per square foot per 30 days.

$$
\begin{align*}
\textrm{complaints}_{b,t} & \sim \textrm{Poisson}(\textrm{sq. foot}_b\,\lambda_{b,t}) \\
\lambda_{b,t} & = \exp{(\eta_{b,t} )} \\
\eta_{b,t} &= \alpha + \beta \, \textrm{traps}_{b,t}
\end{align*}
$$

The term $\text{sq. foot}$ is called an **exposure term**. If we log the term, we can put it in $\eta_{b,t}$:

$$
\begin{align*}
\textrm{complaints}_{b,t} & \sim \textrm{Poisson}(\lambda_{b,t}) \\
\lambda_{b,t} & = \exp{(\eta_{b,t} )} \\
\eta_{b,t} &= \alpha + \beta \, \textrm{traps}_{b,t} + \textrm{log sq. foot}_b
\end{align*}
$$

A quick test shows us that there appears to be a relationship between the square footage of the building and the number of complaints received:


In [ ]:
#| warning: false
plt.clf()
sns.regplot(x=np.log(pest_data.total_sq_foot),y=np.log1p(pest_data.complaints))
plt.show()
plt.close()

Using the property manager's intuition, we include two extra pieces of information we know about the building - the (log of the) square floor space and whether there is a live in super or not - into both the simulated and real data.


In [ ]:
stan_dat_simple['log_sq_foot'] = np.log(pest_data.total_sq_foot/1e4)
stan_dat_simple['live_in_super'] = pest_data.live_in_super

### Stan program for Poisson multiple regression

Now we need a new Stan model that uses multiple predictors.


In [ ]:
#| output: false
multiple_poisson_regression = '''
functions {
    /*
    * Alternative to poisson_log_rng() that 
    * avoids potential numerical problems during warmup
    */
    int poisson_log_safe_rng(real eta) {
    real pois_rate = exp(eta);
    if (pois_rate >= exp(20.79))
      return -9;
    return poisson_rng(pois_rate);
    }
}
data {
    int<lower=1> N;
    array[N] int<lower=0> complaints;
    vector<lower=0>[N] traps;
    vector<lower=0,upper=1>[N] live_in_super;
    vector[N] log_sq_foot;  // exposure term
}
parameters {
    real alpha;
    real beta;
    real beta_super;
}
model {
    beta ~ normal(-0.25, 1);
    beta_super ~ normal(-0.5, 1);
    alpha ~ normal(log(4), 1);
    
    complaints ~ poisson_log(alpha + beta * traps + beta_super * live_in_super + log_sq_foot);
} 
generated quantities {
    array[N] int y_rep;
    
    for (n in 1:N) 
    y_rep[n] = poisson_log_safe_rng(alpha + beta * traps[n] + beta_super * live_in_super[n] + log_sq_foot[n]);
}
'''

multiple_poisson_regression_model = StanModel('../stan_models/multiple_poisson_regression', multiple_poisson_regression)

### Simulate fake data with multiple predictors


In [ ]:
#| output: false
multiple_poisson_regression_dgp = '''
data {
    int<lower=1> N;
}
model {
} 
generated quantities {
    vector[N] log_sq_foot;
    array[N] int live_in_super;
    array[N] int traps;
    array[N] int complaints;
    
    real alpha = normal_rng(log(4), .1);
    real beta = normal_rng(-0.25, .1);
    real beta_super = normal_rng(-0.5, .1);
    
    for (n in 1:N) {
    log_sq_foot[n] = normal_rng(1.5, .1);
    live_in_super[n] = bernoulli_rng(0.5);
    traps[n] = poisson_rng(8);
    complaints[n] = poisson_log_rng(alpha + log_sq_foot[n] 
                               + beta * traps[n] 
                               + beta_super * live_in_super[n]);
    }
}
'''

multiple_poisson_regression_dgp_model = StanModel('../stan_models/multiple_poisson_regression_dgp', multiple_poisson_regression_dgp)

Fit / sample the prior predictive check model:


In [ ]:
#| output: false
fitted_model_dgp = multiple_poisson_regression_dgp_model.sample(
    data={'N': len(pest_data.traps)},
    chains = 1,
    iter_sampling = 1,
    show_progress = False,
    fixed_param = True
)

samps_dgp = az.from_cmdstanpy(fitted_model_dgp)

Now pop that simulated data into a dictionary ready for Stan.


In [ ]:
stan_dat_fake = {
    'N': len(pest_data.traps),
    'complaints': samps_dgp.posterior.complaints.sel(chain=0, draw=0).to_numpy().astype(int),
    'traps': samps_dgp.posterior.traps.sel(chain=0, draw=0),
    'live_in_super': samps_dgp.posterior.live_in_super.sel(chain=0, draw=0),
    'log_sq_foot': samps_dgp.posterior.log_sq_foot.sel(chain=0, draw=0)
}

And then compile and fit the model we wrote for the multiple regression.


In [ ]:
#| output: false
fit_model_P_mult = multiple_poisson_regression_model.sample(data=stan_dat_fake, show_progress=False)
posterior_alpha_beta = az.extract(az.from_cmdstanpy(fit_model_P_mult), 'posterior', var_names=['alpha', 'beta', 'beta_super']).to_dataframe()[['alpha', 'beta', 'beta_super']]

Then compare these parameters to the true parameters:


In [ ]:
true_alpha = samps_dgp.posterior.alpha.sel(chain=0, draw=0).to_numpy()
true_beta = samps_dgp.posterior.beta.sel(chain=0, draw=0).to_numpy()
true_beta_super = samps_dgp.posterior.beta_super.sel(chain=0, draw=0).to_numpy()

plt.clf()
_, ax = plt.subplots(1,3, figsize=(8,4))
sns.histplot(posterior_alpha_beta, x='alpha', bins=30, ax=ax[0])
ax[0].axvline(true_alpha, color='r', linewidth=2)
sns.histplot(posterior_alpha_beta, x='beta', bins=30, ax=ax[1])
ax[1].axvline(true_beta, color='r', linewidth=2)
sns.histplot(posterior_alpha_beta, x='beta_super', bins=30, ax=ax[2])
ax[2].axvline(true_beta_super, color='r', linewidth=2)
plt.show()
plt.close()

We've recovered the parameters sufficiently well, so we've probably coded the Stan program correctly and we're ready to fit the real data.

### Fit the real data

Now let's use the real data and explore the fit.


In [ ]:
#| output: false
fit_model_P_mult_real = multiple_poisson_regression_model.sample(data=stan_dat_simple, show_progress=False)
az_fit_P_mult_real_data = az.from_cmdstanpy(fit_model_P_mult_real, posterior_predictive='y_rep')
y_rep = az_fit_P_mult_real_data.posterior_predictive.y_rep

In [ ]:
plt.clf()
for i in range(200):
    sns.kdeplot(y_rep.sel(chain=slice(0), draw=i).to_dataframe(), x='y_rep', color='k', alpha=0.2)
sns.kdeplot(stan_dat_simple['complaints'], color='r', linewidth=2)
plt.xlim(0,25)
plt.show()
plt.close()

This again looks like we haven't captured the smaller counts very well, nor have we captured the larger counts.


In [ ]:
plt.clf()
_, ax = plt.subplots()
sns.histplot(np.mean(y_rep == 0, axis=2).to_numpy().flatten(), bins=20, ax=ax)
ax.axvline(np.mean(stan_dat_simple['complaints'] == 0), linewidth=2, color='r')
plt.show()
plt.close()

We're still severely underestimating the proportion of zeros in the data. Ideally this vertical line would fall somewhere within the histogram.

We can also plot uncertainty intervals for the predicted complaints for different numbers of traps.


In [ ]:
plt.clf()
_, ax = plt.subplots()

ax.plot(stan_dat_simple['traps'], stan_dat_simple['complaints'], 'o', fillstyle='none')

grouped_indices = defaultdict(list)
for idx, trap in enumerate(stan_dat_simple['traps']):
    grouped_indices[trap].append(idx)

grouped_array = {trap: y_rep[:, :, idx_list].to_numpy().flatten() for trap, idx_list in grouped_indices.items()}

for traps_x, complaints_y in grouped_array.items():
    inner_prob = np.array([np.quantile(complaints_y, 0.75), np.quantile(complaints_y, 0.25)])  # az.hdi(complaints_y, hdi_prob=0.5)
    outer_prob = np.array([np.quantile(complaints_y, 0.95), np.quantile(complaints_y, 0.05)])  # az.hdi(complaints_y, hdi_prob=0.9)
    ax.plot(np.full(inner_prob.shape, traps_x), inner_prob, 'k', linewidth=2)
    ax.plot(np.full(outer_prob.shape, traps_x), outer_prob, 'k', linewidth=0.5)
    ax.plot(traps_x, np.mean(complaints_y), 'ko')

plt.show()
plt.close()

We can see that we've increased the tails a bit more at the larger numbers of traps but we still have some large observed numbers of complaints that the model would consider extremely unlikely events.

## Modeling count data with the Negative Binomial

When we considered modelling the data using a Poisson, we saw that the model didn't appear to fit as well to the data as we would like. In particular the model under-predicted low and high numbers of complaints, and over-predicted the medium number of complaints. This is one indication of **over-dispersion**, where the variance is larger than the mean. A Poisson model doesn't fit over-dispersed count data very well because the same parameter $\lambda$, controls both the expected counts and the variance of these counts. The natural alternative to this is the **negative binomial model**:

$$
\begin{align*}
\text{complaints}_{b,t} & \sim \text{Neg-Binomial}(\lambda_{b,t}, \phi) \\
\lambda_{b,t} & = \exp{(\eta_{b,t})} \\
\eta_{b,t} &= \alpha + \beta \, {\rm traps}_{b,t} + \beta_{\rm super} \, {\rm super}_{b} + \text{log_sq_foot}_{b}
\end{align*}
$$

In Stan the negative binomial mass function we'll use is called $\texttt{neg_binomial_2_log}(\text{ints} \, y, \text{reals} \, \eta, \text{reals} \, \phi)$ in Stan. Like the `poisson_log` function, this negative binomial mass function that is parameterized in terms of its log-mean, $\eta$, but it also has a precision $\phi$ such that

$$
\mathbb{E}[y] \, = \lambda = \exp(\eta)
$$

$$
\text{Var}[y] = \lambda + \lambda^2/\phi = \exp(\eta) + \exp(\eta)^2 / \phi.
$$

As $\phi$ gets larger the term $\lambda^2 / \phi$ approaches zero and so the variance of the negative-binomial approaches $\lambda$, i.e., the negative-binomial gets closer and closer to the Poisson.

### Stan program for negative-binomial regression


In [ ]:
#| output: false
multiple_NB_regression = '''
functions {
    /*
    * Alternative to neg_binomial_2_log_rng() that 
    * avoids potential numerical problems during warmup
    */
    int neg_binomial_2_log_safe_rng(real eta, real phi) {
    real gamma_rate = gamma_rng(phi, phi / exp(eta));
    if (gamma_rate >= exp(20.79))
      return -9;
      
    return poisson_rng(gamma_rate);
    }
}
data {
    int<lower=1> N;
    vector<lower=0>[N] traps;
    vector<lower=0,upper=1>[N] live_in_super;
    vector[N] log_sq_foot;
    array[N] int<lower=0> complaints;
}
parameters {
    real alpha;
    real beta;
    real beta_super;
    real<lower=0> inv_phi;  // Enforce positivity
}
transformed parameters {
    real phi = inv(inv_phi);
}
model {
    alpha ~ normal(log(4), 1);
    beta ~ normal(-0.25, 1);
    beta_super ~ normal(-0.5, 1);
    inv_phi ~ normal(0, 1); // Alternative: normal(0.1, 0.1) // Alternative: exponential(1)
    
    complaints ~ neg_binomial_2_log(alpha + beta * traps + beta_super * live_in_super
                                  + log_sq_foot, phi);
} 
generated quantities {
    array[N] int y_rep;
    for (n in 1:N) 
    y_rep[n] = neg_binomial_2_log_safe_rng(alpha + beta * traps[n] + beta_super * live_in_super[n]
                                       + log_sq_foot[n], phi);
}
'''

multiple_NB_regression_model = StanModel('../stan_models/multiple_NB_regression', multiple_NB_regression)

### Fake data fit: Multiple NB regression


In [ ]:
#| output: false
multiple_NB_regression_dgp = '''
data {
    int<lower=1> N;
}
model {
} 
generated quantities {
    vector[N] log_sq_foot;
    array[N] int live_in_super;
    array[N] int traps;
    array[N] int complaints;
    real alpha = normal_rng(log(4), .1);
    real beta = normal_rng(-0.25, .1);
    real beta_super = normal_rng(-0.5, .1);
    real inv_phi = abs(normal_rng(0,1));
    
    for (n in 1:N) {
    log_sq_foot[n] = normal_rng(1.5, .1);
    live_in_super[n] = bernoulli_rng(0.5);
    traps[n] = poisson_rng(8);
    complaints[n] = neg_binomial_2_log_rng(alpha + log_sq_foot[n] 
                               + beta * traps[n] 
                               + beta_super * live_in_super[n], inv(inv_phi));
    }
}
'''

multiple_NB_regression_dpg_model = StanModel('../stan_models/multiple_NB_regression_dgp', multiple_NB_regression_dgp)

We're going to generate one draw from the fake data model so we can use the data to fit our model and compare the known values of the parameters to the posterior density of the parameters.


In [ ]:
#| output: false
fitted_model_dgp_NB = multiple_NB_regression_dpg_model.sample(
    data={'N': len(pest_data.traps)},
    chains = 1,
    iter_sampling = 1,
    show_progress = False,
    fixed_param=True
)

samps_dgp_NB = az.from_cmdstanpy(fitted_model_dgp_NB)

Create a dictionary to feed into the Stan model.


In [ ]:
stan_dat_fake_NB = {
    'N': len(pest_data.traps),
    'traps': samps_dgp_NB.posterior.traps.sel(chain=0, draw=0),
    'complaints': samps_dgp_NB.posterior.complaints.sel(chain=0, draw=0).to_numpy().astype(int),
    'live_in_super': samps_dgp_NB.posterior.live_in_super.sel(chain=0, draw=0),
    'log_sq_foot': samps_dgp_NB.posterior.log_sq_foot.sel(chain=0, draw=0)
}

Now we run our NB regression over the fake data and extract the samples to examine posterior predictive checks and to check whether we've sufficiently recovered our known parameters, $\text{alpha}$ $\texttt{beta}$.


In [ ]:
#| output: false
fitted_model_NB = multiple_NB_regression_model.sample(data=stan_dat_fake_NB, show_progress=False)
posterior_alpha_beta_NB = (
    az.extract(
        az.from_cmdstanpy(fitted_model_NB), 'posterior'
    ).to_dataframe()[['alpha', 'beta', 'beta_super', 'inv_phi']]
)

Construct the vector of true values from your simulated dataset and compare to the recovered parameters.


In [ ]:
true_params = ['alpha', 'beta', 'beta_super', 'inv_phi']

plt.clf()
_, axes = plt.subplots(2,2, figsize=(6,4))
for i, ax in enumerate(axes.flat):
    sns.histplot(posterior_alpha_beta_NB, x=true_params[i], bins=30, ax=ax)
    ax.axvline(samps_dgp_NB.posterior[true_params[i]].sel(chain=0, draw=0).to_numpy(), color='r', linewidth=2)
    ax.set(ylabel='')
    ax.yaxis.set_ticks([])
plt.show()
plt.close()

### Fit to real data and check the fit


In [ ]:
#| output: false
fitted_model_NB = multiple_NB_regression_model.sample(data=stan_dat_simple, show_progress=False)
az_fitted_model_NB_data = az.from_cmdstanpy(fitted_model_NB, posterior_predictive='y_rep')
y_rep = az_fitted_model_NB_data.posterior_predictive.y_rep

Let's look at our predictions vs. the data.


In [ ]:
plt.clf()
for i in range(200):
    sns.kdeplot(y_rep.sel(chain=slice(0), draw=i).to_dataframe(), x='y_rep', color='k', alpha=0.2)
sns.kdeplot(stan_dat_simple['complaints'], color='r', linewidth=2)
plt.xlim(0,)
plt.show()
plt.close()

It appears that our model now captures both the number of small counts better as well as the tails.

Let's check if the negative binomial model does a better job capturing the number of zeros:


In [ ]:
plt.clf()
_, ax = plt.subplots()
sns.histplot(np.mean(y_rep == 0, axis=2).to_numpy().flatten(), bins=25, ax=ax)
ax.axvline(np.mean(stan_dat_simple['complaints'] == 0), linewidth=2, color='r')
plt.show()
plt.close()

These look OK, but let's look at the standardized residual plot.


In [ ]:
mean_inv_phi = az_fitted_model_NB_data.posterior.inv_phi.mean(dim=['chain', 'draw'])
mean_y_rep = y_rep.mean(dim=['chain', 'draw'])
std_resid = (stan_dat_simple['complaints'] - mean_y_rep) / np.sqrt(mean_y_rep + mean_y_rep**2 * mean_inv_phi)

plt.clf()
plt.plot(mean_y_rep, std_resid, 'o', alpha=0.5)
plt.axhline(2, linestyle='--')
plt.axhline(-2, linestyle='--')
plt.xlabel('mean_y_rep')
plt.ylabel('std_resid')
plt.show()
plt.close()

Looks OK, but we still have some very large *standardized* residuals. This might be because we are currently ignoring that the data are clustered by buildings, and that the probability of roach issue may vary substantially across buildings.


In [ ]:
plt.clf()
fig, ax = plt.subplots()
vals, counts = np.unique(y_rep, return_counts=True)
hist_range = (min(vals), max(vals))
az.plot_dist(stan_dat_simple['complaints'], kind='hist', ax=ax, 
             hist_kwargs={'bins': np.arange(hist_range[0], hist_range[1] + 1, 1), 'range': hist_range})

secax = ax.twinx()
secax.plot(vals, counts, color='red', linewidth=1.5)

ax.set_xlim(hist_range[0]-0.5, hist_range[1]+0.5)

secax.set_yticklabels([])
secax.yaxis.set_ticks([])

ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=5))

plt.show()
plt.close()

The rootogram now looks much more plausible. We can tell this because now the expected number of complaints matches much closer to the observed number of complaints. However, we still have some larger counts that appear to be outliers for the model.

Check predictions by number of traps:


In [ ]:
plt.clf()
_, ax = plt.subplots()

ax.plot(stan_dat_simple['traps'], stan_dat_simple['complaints'], 'o', fillstyle='none')

grouped_indices = defaultdict(list)
for idx, trap in enumerate(stan_dat_simple['traps']):
    grouped_indices[trap].append(idx)

grouped_array = {trap: y_rep[:, :, idx_list].to_numpy().flatten() for trap, idx_list in grouped_indices.items()}

for traps_x, complaints_y in grouped_array.items():
    inner_prob = np.array([np.quantile(complaints_y, 0.75), np.quantile(complaints_y, 0.25)])  # az.hdi(complaints_y, hdi_prob=0.5)
    outer_prob = np.array([np.quantile(complaints_y, 0.95), np.quantile(complaints_y, 0.05)])  # az.hdi(complaints_y, hdi_prob=0.9)
    ax.plot(np.full(inner_prob.shape, traps_x), inner_prob, 'k', linewidth=2)
    ax.plot(np.full(outer_prob.shape, traps_x), outer_prob, 'k', linewidth=0.5)
    ax.plot(traps_x, np.mean(complaints_y), 'ko')

plt.show()
plt.close()

We haven't used the fact that the data are clustered by building yet. A posterior predictive check might elucidate whether it would be a good idea to add the building information into the model.


In [ ]:
plt.clf()

grouped_indices = defaultdict(list)
for idx, building_id in enumerate(pest_data['building_id']):
    grouped_indices[building_id].append(idx)

grouped_array = {bldg_id: y_rep[:, :, idx_list].mean(axis=2).to_numpy().flatten() for bldg_id, idx_list in grouped_indices.items()}

_, axes = plt.subplots(2, 5, figsize=(7,4))

for ax, key in zip(axes.flat, sorted(grouped_array.keys())):
    sns.histplot(grouped_array[key], bins=30, ax=ax)
    ax.axvline(pest_data[pest_data['building_id']==key].complaints.mean(), color='r', linewidth=2)
    ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=4))
    ax.set(ylabel='', title=f"Bulding = {key}")
    ax.yaxis.set_ticks([])

plt.show()
plt.close()

We're getting plausible predictions for most building means but some are estimated better than others and some have larger uncertainties than we might expect. If we explicitly model the variation across buildings we may be able to get much better estimates.

### Garbage Collect


In [ ]:
exceptions = ['pest_data', 'exceptions', 'active_variables']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (CmdStanModel, cmdstanpy.CmdStanMCMC, plt.Axes, 
                           az.InferenceData, pd.DataFrame, pd.Series, 
                           dict, list, int, float, str, tuple, plt.Figure, defaultdict,
                           np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var, _, g, y_rep, mean_inv_phi, mean_y_rep

gc.collect()

## Hierarchical modeling

### Modeling varying intercepts for each building

Let's add a hierarchical intercept parameter, $\alpha_b$ at the building level to our model.

$$
\begin{align*}
\text{complaints}_{b,t} &\sim \text{Neg-Binomial}(\lambda_{b,t}, \phi) \\
\lambda_{b,t}  &= \exp{(\eta_{b,t})} \\
\eta_{b,t} &= \mu_b + \beta \, {\rm traps}_{b,t} + \beta_{\rm super}\, {\rm super}_b + \text{log_sq_foot}_b \\
\mu_b &\sim \text{Normal}(\alpha, \sigma_{\mu})
\end{align*}
$$

In our Stan model, $\mu_b$ is the $b$-th element of the vector $\texttt{mu}$ which has one element per building.

One of our predictors varies only by building, so we can rewrite the above model more efficiently like so:

$$
\begin{align*}
\eta_{b,t} &= \mu_b + \beta \, {\rm traps}_{b,t} + \text{log_sq_foot}_b\\
\mu_b &\sim \text{Normal}(\alpha +  \beta_{\text{super}} \, \text{super}_b , \sigma_{\mu})
\end{align*}
$$

We have more information at the building level as well, like the average age of the residents, the average age of the buildings, and the average per-apartment monthly rent so we can add that data into a matrix called `building_data`, which will have one row per building and four columns:

-   `live_in_super`
-   `age_of_building`
-   `average_tentant_age`
-   `monthly_average_rent`

We'll write the Stan model like:

$$
\begin{align*}
\eta_{b,t} &= \alpha_b + \beta \, {\rm traps} + \text{log_sq_foot}\\
\mu &\sim \text{Normal}(\alpha + \texttt{building_data} \, \zeta, \,\sigma_{\mu})
\end{align*}
$$

### Prepare building data for hierarchical

We'll need to do some more data prep before we can fit our models. Firstly to use the building variable in Stan we will need to transform it from a factor variable to an integer variable.


In [ ]:
N_months = len(pest_data['date'].unique())
N_buildings = len(pest_data['building_id'].unique())

# Add some IDs for building and month
pest_data['building_fac'] = pd.Categorical(pest_data['building_id'], categories=pest_data['building_id'].unique())
pest_data['building_idx'] = pest_data['building_fac'].cat.codes + 1  # Convert to integer index (1-based)
pest_data['ids'] = list(range(1, N_months + 1)) * N_buildings
pest_data['mo_idx'] = pest_data['date'].dt.month  # Extract month from date column

# Center and rescale the building specific data
building_data = pest_data[[
    'building_idx',
    'live_in_super',
    'age_of_building',
    'total_sq_foot',
    'average_tenant_age',
    'monthly_average_rent'
]].drop_duplicates().sort_values('building_idx').drop(columns=['building_idx'])

# Centering without scaling (subtract mean but don't divide by std)
building_data = building_data - building_data.mean()

# Scale by constants
building_data['age_of_building'] /= 10
building_data['total_sq_foot'] /= 10000
building_data['average_tenant_age'] /= 10
building_data['monthly_average_rent'] /= 1000

# Convert to NumPy matrix
building_data_matrix = building_data.to_numpy()

# Make data dictionary for Stan
stan_dat_hier = {
  'complaints': pest_data.complaints,
  'traps': pest_data.traps,
  'N': len(pest_data.traps),
  'J': N_buildings,
  'M': N_months,
  'log_sq_foot': np.log(pest_data.total_sq_foot/1e4),
  'building_data': np.delete(building_data_matrix, 2, axis=1),
  'mo_idx': pest_data.mo_idx,
  'K': 4,
  'building_idx': pest_data.building_idx
  }

### Compile and fit the hierarchical model

Let's code up and compile the model.


In [ ]:
#| output: false
hier_NB_regression = '''
functions {
  /*
  * Alternative to neg_binomial_2_log_rng() that 
  * avoids potential numerical problems during warmup
  */
  int neg_binomial_2_log_safe_rng(real eta, real phi) {
    real gamma_rate = gamma_rng(phi, phi / exp(eta));
    if (gamma_rate >= exp(20.79))
      return -9;
      
    return poisson_rng(gamma_rate);
  }
}
data {
  int<lower=1> N;                     
  array[N] int<lower=0> complaints;
  vector<lower=0>[N] traps;
  
  // 'exposure'
  vector[N] log_sq_foot;  
  
  // building-level data
  int<lower=1> K;
  int<lower=1> J;
  array[N] int <lower=1, upper=J> building_idx;
  matrix[J,K] building_data;
}
parameters {
  real<lower=0> inv_phi;   // 1/phi (easier to think about prior for 1/phi instead of phi)
  real beta;               // coefficient on traps
  
  vector[J] mu;            // buildings-specific intercepts
  real<lower=0> sigma_mu;  // sd of building-specific intercepts
  real alpha;              // intercept of model for mu
  vector[K] zeta;          // coefficients on building-level predictors in model for mu 
}
transformed parameters {
  real phi = inv(inv_phi);
}
model {
  mu ~ normal(alpha + building_data * zeta, sigma_mu);
  sigma_mu ~ normal(0, 1);
  alpha ~ normal(log(4), 1);
  zeta ~ normal(0, 1);  // could also use informative priors on the different elements
  beta ~ normal(-0.25, 1);
  inv_phi ~ normal(0, 1);
  
  complaints ~ neg_binomial_2_log(mu[building_idx] + beta * traps + log_sq_foot, phi);
} 
generated quantities {
  array[N] int y_rep;
  for (n in 1:N) {
    real eta_n = mu[building_idx[n]] + beta * traps[n] + log_sq_foot[n];
    y_rep[n] = neg_binomial_2_log_safe_rng(eta_n, phi);
  }
}
'''

hier_NB_regression_model = StanModel('../stan_models/hier_NB_regression', hier_NB_regression)

Fit the model to data.


In [ ]:
#| output: false
fitted_model_NB_hier = hier_NB_regression_model.sample(data=stan_dat_hier, 
                                                       chains=4, 
                                                       parallel_chains=4,
                                                       iter_sampling=4000, 
                                                       seed=[1034992638, 1034992638, 1034992638, 1034992638],
                                                       show_progress=False)
az_samps_hier_NB = az.from_cmdstanpy(fitted_model_NB_hier, posterior_predictive='y_rep', observed_data=stan_dat_hier)

### Diagnostics

We get a bunch of warnings from Stan about divergent transitions, which is an indication that there may be regions of the posterior that have not been explored by the Markov chains.

Divergences are discussed in more detail in the course slides as well as the **bayesplot** [MCMC diagnostics vignette](http://mc-stan.org/bayesplot/articles/visual-mcmc-diagnostics.html) and [*A Conceptual Introduction to Hamiltonian Monte Carlo*](https://arxiv.org/abs/1701.02434).

In this example we will see that we have divergent transitions because we need to reparameterize our model - i.e., we will retain the overall structure of the model, but transform some of the parameters so that it is easier for Stan to sample from the parameter space. Before we go through exactly how to do this reparameterization, we will first go through what indicates that this is something that reparameterization will resolve. We will go through:

1.  Examining the fitted parameter values, including the effective sample size
2.  Traceplots and scatterplots that reveal particular patterns in locations of the divergences.

Let us first run the `diagnose` and `summary` method on the model object `fitted_model_NB_hier`:


In [ ]:
print(fitted_model_NB_hier.diagnose())

This reports that 32% of the samples ended with a divergence.

Now, let's extract the fits from the model.


In [ ]:
summary = fitted_model_NB_hier.summary(percentiles=(5, 95)).round(2)
summary[~summary.index.str.contains('y_rep')]

You can see that the effective samples are quite low for many of the parameters relative to the total number of samples. This alone isn't indicative of the need to reparameterize, but it indicates that we should look further at the trace plots and pairs plots. First let's look at the traceplots to see if the divergent transitions form a pattern.


In [ ]:
plt.clf()
az.plot_trace(az_samps_hier_NB, var_names='sigma_mu', divergences='bottom')
plt.show()
plt.close()

Looks as if the divergent parameters, the the black bars underneath the traceplots correspond to samples where the sampler gets stuck at one parameter value for $\sigma_\mu$.


In [ ]:
plot_mu = az_samps_hier_NB.posterior.mu.sel(mu_dim_0=3).to_numpy().flatten()
plot_sigma_mu = np.log(az_samps_hier_NB.posterior.sigma_mu.to_numpy().flatten())
divergent_filter = az_samps_hier_NB.sample_stats.diverging.to_numpy().flatten()

plt.clf()
_, ax = plt.subplots()
sns.scatterplot(x=plot_mu, y=plot_sigma_mu, alpha=0.2, ax=ax)
sns.scatterplot(x=plot_mu[divergent_filter], y=plot_sigma_mu[divergent_filter], color='r', alpha=0.5, ax=ax)
plt.show()
plt.close()

What we have here is a cloud-like shape, with most of the divergences clustering towards the bottom. We'll see a bit later that we actually want this to look more like a funnel than a cloud, but the divergences are indicating that the sampler can't explore the narrowing neck of the funnel.

One way to see why we should expect some version of a funnel is to look at some simulations from the prior, which we can do without MCMC and thus with no risk of sampling problems:


In [ ]:
N_sims = 1000
log_sigma = np.random.normal(loc=0, scale=1, size=N_sims)
theta = np.random.normal(loc=0, scale=np.exp(log_sigma))

plt.clf()
_, ax = plt.subplots()
sns.scatterplot(x=theta, y=log_sigma, alpha=0.2, ax=ax)
plt.show()
plt.close()

Of course, if the data is at all informative we shouldn't expect the posterior to look exactly like the prior. But unless the data is incredibly informative about the parameters and the posterior concentrates away from the narrow neck of the funnel, the sampler is going to have to confront the funnel geometry. (See the [Visual MCMC Diagnostics](http://mc-stan.org/bayesplot/articles/visual-mcmc-diagnostics.html) vignette for more on this.)

Another way to look at the divergences is via a parallel coordinates plot:


In [ ]:
param_mat = np.zeros((11, 4000*4))
param_mat[0,:] = az_samps_hier_NB.posterior.sigma_mu.to_numpy().flatten()
for i in range(10):
    param_mat[i+1,:] = az_samps_hier_NB.posterior.mu.sel(mu_dim_0=i).to_numpy().flatten()

divergent_mat = np.zeros((11, np.sum(divergent_filter)))
for i in range(11):
    divergent_mat[i,:] = param_mat[i, divergent_filter]

custom_labels = ["sigma_mu", "mu[1]", "mu[2]", "mu[3]", "mu[4]", "mu[5]",
                 "mu[6]", "mu[7]", "mu[8]", "mu[9]", "mu[10]"]

plt.clf()
plt.plot(param_mat, 'k', alpha=0.015)
plt.plot(divergent_mat, 'r', alpha=0.025)
plt.xlim(0,10)
plt.xticks(np.arange(11), custom_labels)
plt.show()
plt.close()

Again, we see evidence that our problems concentrate when $\texttt{sigma_mu}$ is small.

### Reparameterize and recheck diagnostics

Instead, we should use the non-centered parameterization for $\mu_b$. We define a vector of auxiliary variables in the parameters block, $\texttt{mu_raw}$ that is given a $\text{Normal}(0, 1)$ prior in the model block. We then make $\texttt{mu}$ a transformed parameter:

We can reparameterize the random intercept $\mu_b$, which is distributed:

$$
\mu_b \sim \text{Normal}(\alpha + \texttt{building_data} \, \zeta, \sigma_{\mu})
$$

```         
transformed parameters {
  vector[J] mu;
  mu = alpha + building_data * zeta + sigma_mu * mu_raw;
}
```

This gives $\texttt{mu}$ a $\text{Normal}(\alpha + \texttt{building_data}\, \zeta, \sigma_\mu)$ distribution, but it decouples the dependence of the density of each element of $\texttt{mu}$ from $\texttt{sigma_mu}$ ($\sigma_\mu$). `hier_NB_regression_ncp` uses the non-centered parameterization for $\texttt{mu}$. We will examine the effective sample size of the fitted model to see whether we've fixed the problem with our reparameterization.

Compile the model.


In [ ]:
#| output: false
hier_NB_regression_ncp = '''
functions {
    /*
    * Alternative to neg_binomial_2_log_rng() that 
    * avoids potential numerical problems during warmup
    */
    int neg_binomial_2_log_safe_rng(real eta, real phi) {
    real gamma_rate = gamma_rng(phi, phi / exp(eta));
    if (gamma_rate >= exp(20.79))
      return -9;
      
    return poisson_rng(gamma_rate);
    }
}
data {
    int<lower=1> N;                     
    array[N] int<lower=0> complaints;
    vector<lower=0>[N] traps;
    
    // 'exposure'
    vector[N] log_sq_foot;
    
    // building-level data
    int<lower=1> K;
    int<lower=1> J;
    array[N] int<lower=1, upper=J> building_idx;
    matrix[J,K] building_data;
}
parameters {
    real<lower=0> inv_phi;   // 1/phi (easier to think about prior for 1/phi instead of phi)
    real beta;               // coefficient on traps
    
    vector[J] mu_raw;        // N(0,1) params for non-centered parameterization of building intercepts 
    real<lower=0> sigma_mu;  // sd of building-specific intercepts
    real alpha;              // intercept of model for mu
    vector[K] zeta;          // coefficients on building-level predictors in model for mu 
}
transformed parameters {
    real phi = inv(inv_phi);
    
    // non-centered parameterization
    vector[J] mu = alpha + building_data * zeta + sigma_mu * mu_raw;
}
model {
    mu_raw ~ normal(0, 1);   // implies mu ~ normal(alpha + building_data * zeta, sigma_mu)
    sigma_mu ~ normal(0, 1);
    alpha ~ normal(log(4), 1);
    zeta ~ normal(0, 1);
    beta ~ normal(-0.25, 1);
    inv_phi ~ normal(0, 1);
    
    complaints ~ neg_binomial_2_log(mu[building_idx] + beta * traps + log_sq_foot, phi);
} 
generated quantities {
    array[N] int y_rep;
    for (n in 1:N) {
        real eta_n = mu[building_idx[n]] + beta * traps[n] + log_sq_foot[n];
        y_rep[n] = neg_binomial_2_log_safe_rng(eta_n, phi);
    }
}
'''

hier_NB_regression_ncp_model = StanModel('../stan_models/hier_NB_regression_ncp', hier_NB_regression_ncp)

Fit the model to the data.


In [ ]:
#| output: false
fitted_model_NB_hier_ncp = hier_NB_regression_ncp_model.sample(
    data=stan_dat_hier,
    chains=4,
    parallel_chains=4,
    adapt_delta=0.95,
    show_progress=False
)
az_samps_hier_NB_hier_ncp = az.from_cmdstanpy(fitted_model_NB_hier_ncp, posterior_predictive='y_rep', observed_data=stan_dat_hier)

Examining the fit of the new model


In [ ]:
print(fitted_model_NB_hier_ncp.diagnose())

In [ ]:
#| warning: false
summary = fitted_model_NB_hier.summary(percentiles=(5, 95)).round(2)
summary[~summary.index.str.contains(r'^(y_rep|zeta)', regex=True)]

This has improved the effective sample sizes of $\texttt{mu}$. We extract the parameters to run our usual posterior predictive checks.


In [ ]:
plot_mu = az_samps_hier_NB_hier_ncp.posterior.mu.sel(mu_dim_0=3).to_numpy().flatten()
plot_sigma_mu = np.log(az_samps_hier_NB_hier_ncp.posterior.sigma_mu.to_numpy().flatten())
divergent_filter = az_samps_hier_NB_hier_ncp.sample_stats.diverging.to_numpy().flatten()

plt.clf()
_, ax = plt.subplots()
sns.scatterplot(x=plot_mu, y=plot_sigma_mu, alpha=0.2, ax=ax)
sns.scatterplot(x=plot_mu[divergent_filter], y=plot_sigma_mu[divergent_filter], color='r', alpha=0.5, ax=ax)
ax.set(xlabel='mu[4]', ylabel='log(sigma_mu)')
plt.show()
plt.close()

In [ ]:
plt.clf()
az.plot_parallel(az_samps_hier_NB_hier_ncp, 
                 var_names=['sigma_mu', 'mu'])
plt.show()
plt.close()

In [ ]:
y_rep = az_samps_hier_NB_hier_ncp.posterior_predictive.y_rep

plt.clf()
for i in range(200):
    sns.kdeplot(y_rep.sel(chain=slice(0), draw=i).to_dataframe(), x='y_rep', color='k', alpha=0.2)
sns.kdeplot(stan_dat_hier['complaints'], color='r', linewidth=2)
plt.xlim(0,)
plt.show()
plt.close()

This looks quite nice. If we've captured the building-level means well, then the posterior distribution of means by building should match well with the observed means of the quantity of building complaints by month.


In [ ]:
plt.clf()

grouped_indices = defaultdict(list)
for idx, building_id in enumerate(pest_data['building_id']):
    grouped_indices[building_id].append(idx)

grouped_array = {bldg_id: y_rep[:, :, idx_list].mean(axis=2).to_numpy().flatten() for bldg_id, idx_list in grouped_indices.items()}

_, axes = plt.subplots(2, 5, figsize=(7,4))

for ax, key in zip(axes.flat, sorted(grouped_array.keys())):
    sns.histplot(grouped_array[key], bins=30, ax=ax)
    ax.axvline(pest_data[pest_data['building_id']==key].complaints.mean(), color='r', linewidth=2)
    ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=4))
    ax.set(ylabel='', title=f"Bulding = {key}")
    ax.yaxis.set_ticks([])

plt.show()
plt.close()

We weren't doing terribly with the building-specific means before, but now they are all well-captured by our model. The model is also able to do a decent job estimating within-building variability:


In [ ]:
plt.clf()

grouped_indices = defaultdict(list)
for idx, building_id in enumerate(pest_data['building_id']):
    grouped_indices[building_id].append(idx)

grouped_array = {bldg_id: y_rep[:, :, idx_list].std(axis=2).to_numpy().flatten() for bldg_id, idx_list in grouped_indices.items()}

_, axes = plt.subplots(2, 5, figsize=(7,4))

for ax, key in zip(axes.flat, sorted(grouped_array.keys())):
    sns.histplot(grouped_array[key], bins=30, ax=ax)
    ax.axvline(pest_data[pest_data['building_id']==key].complaints.std(), color='r', linewidth=2)
    ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=4))
    ax.set(ylabel='', title=f"Bulding = {key}")
    ax.yaxis.set_ticks([])

plt.show()
plt.close()

Predictions by number of traps:


In [ ]:
plt.clf()
_, ax = plt.subplots()

ax.plot(stan_dat_hier['traps'], stan_dat_hier['complaints'], 'o', fillstyle='none')

grouped_indices = defaultdict(list)
for idx, trap in enumerate(stan_dat_hier['traps']):
    grouped_indices[trap].append(idx)

grouped_array = {trap: y_rep[:, :, idx_list].to_numpy().flatten() for trap, idx_list in grouped_indices.items()}

for traps_x, complaints_y in grouped_array.items():
    inner_prob = np.array([np.quantile(complaints_y, 0.75), np.quantile(complaints_y, 0.25)])
    outer_prob = np.array([np.quantile(complaints_y, 0.95), np.quantile(complaints_y, 0.05)])
    ax.plot(np.full(inner_prob.shape, traps_x), inner_prob, 'k', linewidth=2)
    ax.plot(np.full(outer_prob.shape, traps_x), outer_prob, 'k', linewidth=0.5)
    ax.plot(traps_x, np.mean(complaints_y), 'ko')

plt.show()
plt.close()

Standardized residuals:


In [ ]:
mean_inv_phi = az_samps_hier_NB_hier_ncp.posterior.inv_phi.mean(dim=['chain', 'draw'])
mean_y_rep = y_rep.mean(dim=['chain', 'draw'])
std_resid = (stan_dat_hier['complaints'] - mean_y_rep) / np.sqrt(mean_y_rep + mean_y_rep**2 * mean_inv_phi)

plt.clf()
plt.plot(mean_y_rep, std_resid, 'o', alpha=0.5)
plt.axhline(2, linestyle='--')
plt.axhline(-2, linestyle='--')
plt.xlabel('mean_y_rep')
plt.ylabel('std_resid')
plt.show()
plt.close()

Rootogram:


In [ ]:
plt.clf()
fig, ax = plt.subplots()
vals, counts = np.unique(y_rep, return_counts=True)
hist_range = (min(vals), max(vals))
az.plot_dist(stan_dat_hier['complaints'], kind='hist', ax=ax, 
             hist_kwargs={'bins': np.arange(hist_range[0], hist_range[1] + 1, 1), 'range': hist_range})

secax = ax.twinx()
secax.plot(vals, counts, color='red', linewidth=1.5)

ax.set_xlim(hist_range[0]-0.5, hist_range[1]+0.5)

secax.set_yticklabels([])
secax.yaxis.set_ticks([])

ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=5))

plt.show()
plt.close()

### Varying intercepts *and* varying slopes

We've gotten some new data that extends the number of time points for which we have observations for each building. This will let us explore how to expand the model a bit more with varying *slopes* in addition to the varying intercepts and also, later, also model temporal variation.


In [ ]:
import rdata
stan_dat_hier = rdata.read_rds('data/pest_data_longer_stan_dat.RDS')
stan_dat_hier['N'] = stan_dat_hier['N'][0].astype(int)
stan_dat_hier['J'] = stan_dat_hier['J'][0].astype(int)
stan_dat_hier['K'] = stan_dat_hier['K'][0].astype(int)
stan_dat_hier['M'] = stan_dat_hier['M'][0].astype(int)
stan_dat_hier['M_forward'] = stan_dat_hier['M_forward'][0].astype(int)
stan_dat_hier['complaints'] = stan_dat_hier['complaints'].astype(int)
stan_dat_hier['building_idx'] = stan_dat_hier['building_idx'].astype(int)

Perhaps if the levels of complaints differ by building, the coefficient for the effect of traps on building does too. We can add this to our model and observe the fit.

$$
\begin{align*}
\text{complaints}_{b,t} &\sim \text{Neg-Binomial}(\lambda_{b,t}, \phi)  \\
\lambda_{b,t} &= \exp{(\eta_{b,t})}\\
\eta_{b,t} &= \mu_b + \kappa_b \, \texttt{traps}_{b,t} + \text{log_sq_foot}_b \\
\mu_b &\sim \text{Normal}(\alpha + \texttt{building_data} \, \zeta, \sigma_{\mu}) \\
\kappa_b &\sim \text{Normal}(\beta + \texttt{building_data} \, \gamma, \sigma_{\kappa})
\end{align*}
$$

Let's compile the model.


In [ ]:
#| output: false
hier_NB_regression_ncp_slopes_mod = '''
functions {
  /*
  * Alternative to neg_binomial_2_log_rng() that 
  * avoids potential numerical problems during warmup
  */
  int neg_binomial_2_log_safe_rng(real eta, real phi) {
    real gamma_rate = gamma_rng(phi, phi / exp(eta));
    if (gamma_rate >= exp(20.79))
      return -9;
      
    return poisson_rng(gamma_rate);
  }
}
data {
  int<lower=1> N;                     
  array[N] int<lower=0> complaints;
  vector<lower=0>[N] traps;        
  
  // 'exposure'
  vector[N] log_sq_foot;  
  
  // building-level data
  int<lower=1> K;
  int<lower=1> J;
  array[N] int<lower=1, upper=J> building_idx;
  matrix[J,K] building_data;
}
parameters {
  real<lower=0> inv_phi;     // 1/phi (easier to think about prior for 1/phi instead of phi)
  
  vector[J] mu_raw;        // N(0,1) params for non-centered param of building-specific intercepts
  real<lower=0> sigma_mu;  // sd of buildings-specific intercepts
  real alpha;              // 'global' intercept
  vector[K] zeta;          // coefficients on building-level predictors in model for mu
  
  vector[J] kappa_raw;       // N(0,1) params for non-centered param of building-specific slopes
  real<lower=0> sigma_kappa; // sd of buildings-specific slopes
  real beta;                 // 'global' slope on traps variable
  vector[K] gamma;           // coefficients on building-level predictors in model for kappa
}
transformed parameters {
  real phi = inv(inv_phi);
  
  // non-centered parameterization of building-specific intercepts and slopes
  vector[J] mu = alpha + building_data * zeta + sigma_mu * mu_raw;
  vector[J] kappa = beta + building_data * gamma + sigma_kappa * kappa_raw;
}
model {
  inv_phi ~ normal(0, 1);
  
  kappa_raw ~ normal(0,1) ;
  sigma_kappa ~ normal(0, 1);
  beta ~ normal(-0.25, 1);
  gamma ~ normal(0, 1);
  
  mu_raw ~ normal(0,1) ;
  sigma_mu ~ normal(0, 1);
  alpha ~ normal(log(4), 1);
  zeta ~ normal(0, 1);
  
  complaints ~ neg_binomial_2_log(mu[building_idx] + kappa[building_idx] .* traps  + log_sq_foot,
                                  phi);
} 
generated quantities {
  array[N] int y_rep;
  for (n in 1:N) {
    real eta_n = mu[building_idx[n]] + kappa[building_idx[n]] * traps[n] + log_sq_foot[n];
    y_rep[n] = neg_binomial_2_log_safe_rng(eta_n, phi);
  }
}
'''

hier_NB_regression_ncp_slopes_mod_model = StanModel('../stan_models/hier_NB_regression_ncp_slopes_mod', hier_NB_regression_ncp_slopes_mod)

Fit the model to data and extract the posterior draws needed for our posterior predictive checks.


In [ ]:
#| output: false

keys_to_remove = {'pred_mat', 'log_sq_foot_pred', 'M_forward', 'M'}
filtered_stan_dat_hier = {k: v for k, v in stan_dat_hier.items() if k not in keys_to_remove}

fitted_model_NB_hier_slopes = hier_NB_regression_ncp_slopes_mod_model.sample(
    data=filtered_stan_dat_hier,
    chains=4,
    parallel_chains=4,
    adapt_delta=0.95,
    show_progress=False
)

az_samps_NB_hier_slopes = az.from_cmdstanpy(fitted_model_NB_hier_slopes, posterior_predictive='y_rep', observed_data=filtered_stan_dat_hier)

To see if the model infers building-to-building differences in, we can plot a histogram of our marginal posterior distribution for `sigma_kappa`.


In [ ]:
plt.clf()

sns.histplot(az_samps_NB_hier_slopes.posterior.sigma_kappa.to_numpy().flatten(), binwidth=0.005)

plt.show()
plt.close()

In [ ]:
summary = fitted_model_NB_hier_slopes.summary(percentiles=(5, 95)).round(2)
filter_indices = r'^(kappa(\[\d+\])?|beta|alpha|phi|sigma_mu|sigma_kappa|mu(\[\d+\])?)$'
summary[summary.index.str.match(filter_indices, na=False)]

In [ ]:
plt.clf()

sns.histplot(az_samps_NB_hier_slopes.posterior.beta.to_numpy().flatten(), binwidth=0.005)

plt.show()
plt.close()

While the model can't specifically rule out zero from the posterior, it does have mass at small non-zero numbers, so we should leave in the hierarchy over $\texttt{kappa}$. Plotting the marginal data density again, we can see the model still looks well calibrated.


In [ ]:
y_rep = az_samps_NB_hier_slopes.posterior_predictive.y_rep

plt.clf()
for i in range(200):
    sns.kdeplot(y_rep.sel(chain=slice(0), draw=i).to_dataframe(), x='y_rep', color='k', alpha=0.2)
sns.kdeplot(stan_dat_hier['complaints'], color='r', linewidth=2)
plt.xlim(0,)
plt.show()
plt.close()

## Time varying effects and structured priors

We haven't looked at how cockroach complaints change over time. Let's look at whether there's any pattern over time.


In [ ]:
grouped_indices = defaultdict(list)
for idx, month_num in enumerate(stan_dat_hier['mo_idx']):
    if month_num > 12: continue
    grouped_indices[month_num].append(idx)
    
grouped_array = {bldg_id: y_rep[:, :, idx_list].mean(axis=2).to_numpy().flatten() for bldg_id, idx_list in grouped_indices.items()}

_, axes = plt.subplots(3, 4, figsize=(7,4))

for ax, key in zip(axes.flat, sorted(grouped_array.keys())):
    sns.histplot(grouped_array[key], bins=30, ax=ax)
    ax.axvline(stan_dat_hier['complaints'][grouped_indices[key]].mean(), color='r', linewidth=2)
    ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=4))
    ax.set(ylabel='', title=f"Bulding = {key}", xlim=(0, 12))
    ax.yaxis.set_ticks([])

plt.show()
plt.close()

We might augment our model with a log-additive monthly effect, $\texttt{mo}_t$.

$$
\eta_{b,t} = \mu_b + \kappa_b \, \texttt{traps}_{b,t} + \texttt{mo}_t + \text{log_sq_foot}_b
$$

We have complete freedom over how to specify the prior for $\texttt{mo}_t$. There are several competing factors for how the number of complaints might change over time. It makes sense that there might be more roaches in the environment during the summer, but we might also expect that there is more roach control in the summer as well. Given that we're modeling complaints, maybe after the first sighting of roaches in a building, residents are more vigilant, and thus complaints of roaches would increase.

This can be a motivation for using an autoregressive prior for our monthly effects. The model is as follows:

$$
\begin{equation}
\texttt{mo}_t \sim \text{Normal}(\rho \, \texttt{mo}_{t-1}, \sigma_\texttt{mo}) \\
\equiv \\
\texttt{mo}_t = \rho \, \texttt{mo}_{t-1} +\epsilon_t , \quad \epsilon_t \sim \text{Normal}(0, \sigma_\texttt{mo}) \\
\quad \rho \in [-1,1]
\end{equation}
$$

This equation says that the monthly effect in month $t$ is directly related to the last month's monthly effect. Given the description of the process above, it seems like there could be either positive or negative associations between the months, but there should be a bit more weight placed on positive $\rho$s, so we'll put an informative prior that pushes the parameter $\rho$ towards 0.5.

Before we write our prior, however, we have a problem: Stan doesn't implement any densities that have support on $[-1,1]$. We can use variable transformation of a raw variable defined on $[0,1]$ to to give us a density on $[-1,1]$. Specifically,

$$
\begin{equation}
\rho_{\text{raw}} \in [0, 1] \\
\rho = 2 \times \rho_{\text{raw}} - 1
\end{equation}
$$

Then we can put a beta prior on $\rho_\text{raw}$ to push our estimate towards 0.5.

One further wrinkle is that we have a prior for $\texttt{mo}_t$ that depends on $\texttt{mo}_{t-1}$. That is, we are working with the *conditional* distribution of $\texttt{mo}_t$ given $\texttt{mo}_{t-1}$. But what should we do about the prior for $\texttt{mo}_1$, for which we don't have a previous time period in the data?

We need to work out the *marginal* distribution of the first observation. Thankfully we can use the fact that AR models are stationary, so $\text{Var}(\texttt{mo}_t) = \text{Var}(\texttt{mo}_{t-1})$ and $\mathbb{E}(\texttt{mo}_t) = \mathbb{E}(\texttt{mo}_{t-1})$ for all $t$. Therefore the marginal distribution of $\texttt{mo}_1$ is the same as the marginal distribution of any $\texttt{mo}_t$.

First we derive the marginal variance of $\texttt{mo}_{t}$.

$$
\begin{align*}
\text{Var}(\texttt{mo}_t) &= \text{Var}(\rho \texttt{mo}_{t-1} + \epsilon_t)  \\
\text{Var}(\texttt{mo}_t) &= \text{Var}(\rho \texttt{mo}_{t-1}) + \text{Var}(\epsilon_t)
\end{align*}
$$

where the second line holds by independence of $\epsilon_t$ and $\epsilon_{t-1})$. Then, using the fact that $Var(cX) = c^2Var(X)$ for a constant $c$ and the fact that, by stationarity, $\textrm{Var}(\texttt{mo}_{t-1}) = \textrm{Var}(\texttt{mo}_{t})$, we then obtain:

$$
\begin{align*}
\text{Var}(\texttt{mo}_t) &= \rho^2 \text{Var}( \texttt{mo}_{t})  + \sigma_\texttt{mo}^2 \\
\text{Var}(\texttt{mo}_t) &= \frac{\sigma_\texttt{mo}^2}{1 - \rho^2}
\end{align*}
$$

For the mean of $\texttt{mo}_t$ things are a bit simpler:

$$
\begin{align*}
\mathbb{E}(\texttt{mo}_t) &= \mathbb{E}(\rho \, \texttt{mo}_{t-1} + \epsilon_t) \\
\mathbb{E}(\texttt{mo}_t) &= \mathbb{E}(\rho \, \texttt{mo}_{t-1}) + \mathbb{E}(\epsilon_t) \\
\end{align*}
$$

Since $\mathbb{E}(\epsilon_t) = 0$ by assumption we have

$$
\begin{align*}
\mathbb{E}(\texttt{mo}_t) &= \mathbb{E}(\rho \, \texttt{mo}_{t-1})  + 0\\
\mathbb{E}(\texttt{mo}_t) &= \rho \, \mathbb{E}(\texttt{mo}_{t}) \\
\mathbb{E}(\texttt{mo}_t) - \rho \mathbb{E}(\texttt{mo}_t) &= 0  \\
\mathbb{E}(\texttt{mo}_t) &= 0/(1 - \rho)
\end{align*}
$$

which for $\rho \neq 1$ yields $\mathbb{E}(\texttt{mo}_{t}) = 0$.

We now have the marginal distribution for $\texttt{mo}_{t}$, which, in our case, we will use for $\texttt{mo}_1$. The full AR(1) specification is then:

$$
\begin{align*}
\texttt{mo}_1 &\sim \text{Normal}\left(0, \frac{\sigma_\texttt{mo}}{\sqrt{1 - \rho^2}}\right) \\
\texttt{mo}_t &\sim \text{Normal}\left(\rho \, \texttt{mo}_{t-1}, \sigma_\texttt{mo}\right) \forall t > 1
\end{align*}
$$


In [ ]:
#| output: false
hier_NB_regression_ncp_slopes_mod_mos = '''
functions {
  /*
  * Alternative to neg_binomial_2_log_rng() that 
  * avoids potential numerical problems during warmup
  */
  int neg_binomial_2_log_safe_rng(real eta, real phi) {
    real gamma_rate = gamma_rng(phi, phi / exp(eta));
    if (gamma_rate >= exp(20.79))
      return -9;
      
    return poisson_rng(gamma_rate);
  }
}
data {
  int<lower=1> N;   
  array[N] int<lower=0> complaints;
  vector<lower=0>[N] traps;                
  
  // 'exposure'
  vector[N] log_sq_foot;  
  
  // building-level data
  int<lower=1> K;
  int<lower=1> J;
  array[N] int<lower=1, upper=J> building_idx;
  matrix[J,K] building_data;
  
  // month 
  int<lower=1> M; 
  array[N] int<lower=1,upper=M> mo_idx;
}
parameters {
  real<lower=0> inv_phi;   // 1/phi (easier to think about prior for 1/phi instead of phi)
  
  vector[J] mu_raw;        // N(0,1) params for non-centered param of building-specific intercepts
  real<lower=0> sigma_mu;  // sd of buildings-specific intercepts
  real alpha;              // 'global' intercept
  vector[K] zeta;          // coefficients on building-level predictors in model for mu
  
  vector[J] kappa_raw;       // N(0,1) params for non-centered param of building-specific slopes
  real<lower=0> sigma_kappa; // sd of buildings-specific slopes
  real beta;                 // 'global' slope on traps variable
  vector[K] gamma;           // coefficients on building-level predictors in model for kappa
  
  
  vector[M] mo_raw;               // N(0,1) params for non-centered param of AR(1) process
  real<lower=0> sigma_mo;         // sd of month-specific parameters
  real<lower=0,upper=1> rho_raw;  // used to construct AR(1) coefficient
}
transformed parameters {
  real phi = inv(inv_phi);
  
  // non-centered parameterization of building-specific intercepts and slopes
  vector[J] mu = alpha + building_data * zeta + sigma_mu * mu_raw;
  vector[J] kappa = beta + building_data * gamma + sigma_kappa * kappa_raw;
  
  // AR(1) process priors
  real rho = 2.0 * rho_raw - 1.0;
  vector[M] mo = sigma_mo * mo_raw;
  mo[1] /= sqrt(1 - rho^2);
  for (m in 2:M) {
    mo[m] += rho * mo[m-1];
  }
}
model {
  inv_phi ~ normal(0, 1);
  
  kappa_raw ~ normal(0,1) ;
  sigma_kappa ~ normal(0, 1);
  beta ~ normal(-0.25, 1);
  gamma ~ normal(0, 1);
  
  mu_raw ~ normal(0,1) ;
  sigma_mu ~ normal(0, 1);
  alpha ~ normal(log(4), 1);
  zeta ~ normal(0, 1);
  
  mo_raw ~ normal(0,1);
  sigma_mo ~ normal(0, 1);
  rho_raw ~ beta(10, 5);
  
  complaints ~ neg_binomial_2_log(mu[building_idx] + kappa[building_idx] .* traps 
                                 + mo[mo_idx] + log_sq_foot, phi);
}
generated quantities {
  array[N] int y_rep;
  for (n in 1:N) {
    real eta_n = 
      mu[building_idx[n]] + kappa[building_idx[n]] * traps[n] + mo[mo_idx[n]] + log_sq_foot[n];
    y_rep[n] = neg_binomial_2_log_safe_rng(eta_n, phi);
  }
}
'''

comp_model_NB_hier_mos = StanModel('../stan_models/hier_NB_regression_ncp_slopes_mod_mos', hier_NB_regression_ncp_slopes_mod_mos)

In [ ]:
#| output: false
keys_to_remove = {'pred_mat', 'log_sq_foot_pred', 'M_forward'}
filtered_stan_dat_hier = {k: v for k, v in stan_dat_hier.items() if k not in keys_to_remove}

fitted_model_NB_hier_mos = comp_model_NB_hier_mos.sample(
    data=filtered_stan_dat_hier,
    chains=4,
    parallel_chains=4,
    adapt_delta=0.9,
    show_progress=False
)

az_samps_NB_hier_mos = az.from_cmdstanpy(fitted_model_NB_hier_mos, posterior_predictive='y_rep', observed_data=filtered_stan_dat_hier)

In the interest of brevity, we won't go on expanding the model, though we certainly could. What other information would help us understand the data generating process better? What other aspects of the data generating process might we want to capture that we're not capturing now?

As usual, we run through our posterior predictive checks.


In [ ]:
y_rep = az_samps_NB_hier_mos.posterior_predictive.y_rep

plt.clf()
for i in range(200):
    sns.kdeplot(y_rep.sel(chain=slice(0), draw=i).to_dataframe(), x='y_rep', color='k', alpha=0.2)
sns.kdeplot(stan_dat_hier['complaints'], color='r', linewidth=2)
plt.xlim(0,)
plt.show()
plt.close()

In [ ]:
grouped_indices = defaultdict(list)
for idx, month_num in enumerate(stan_dat_hier['mo_idx']):
    if month_num > 12: continue
    grouped_indices[month_num].append(idx)
    
grouped_array = {bldg_id: y_rep[:, :, idx_list].mean(axis=2).to_numpy().flatten() for bldg_id, idx_list in grouped_indices.items()}

_, axes = plt.subplots(3, 4, figsize=(7,4))

for ax, key in zip(axes.flat, sorted(grouped_array.keys())):
    sns.histplot(grouped_array[key], bins=20, ax=ax)
    ax.axvline(stan_dat_hier['complaints'][grouped_indices[key]].mean(), color='r', linewidth=2)
    ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=4))
    ax.set(ylabel='', title=f"Bulding = {key}")
    ax.yaxis.set_ticks([])

plt.show()
plt.close()

As we can see, our monthly random intercept has captured a monthly pattern across all the buildings. We can also compare the prior and posterior for the autoregressive parameter to see how much we've learned. Here are two different ways of comparing the prior and posterior visually:


In [ ]:
import scipy.special as special

# 1) compare draws from prior and draws from posterior
prior_rho = 2 * np.random.beta(a=10, b=5, size=4000) - 1
posterior_rho = az_samps_NB_hier_mos.posterior.rho.to_numpy().flatten()

plt.clf()
sns.histplot(prior_rho, binwidth=0.025)
sns.histplot(posterior_rho, binwidth=0.025)
plt.show()
plt.close()

In [ ]:
# 2) overlay prior density curve on posterior draws
def gen_rho_prior(x):
    alpha, beta = 10, 5
    a, c = -1, 1
    lp = (
        (alpha - 1) * np.log(x - a) +
        (beta - 1) * np.log(c - x) -
        (alpha + beta - 1) * np.log(c - a) -
        special.betaln(3, 4)
    )
    return np.exp(lp)

plt.clf()
_, ax = plt.subplots()
sns.histplot(posterior_rho, binwidth=0.025, ax=ax)
x = np.linspace(-0.99,0.99)
secax = ax.twinx()
secax.plot(x, gen_rho_prior(x), color='k')
secax.set(ylim=(0,0.03))
plt.show()
plt.close()

In [ ]:
summary = fitted_model_NB_hier_mos.summary(percentiles=(5, 95)).round(2)
filter_indices = r'^(gamma(\[\d+\])?|rho|sigma_mu|sigma_kappa)$'
summary[summary.index.str.match(filter_indices, na=False)]

In [ ]:
plt.clf()
_, ax = plt.subplots()

ax.plot(stan_dat_hier['traps'], stan_dat_hier['complaints'], 'o', fillstyle='none')

grouped_indices = defaultdict(list)
for idx, trap in enumerate(stan_dat_hier['traps']):
    grouped_indices[trap].append(idx)

grouped_array = {trap: y_rep[:, :, idx_list].to_numpy().flatten() for trap, idx_list in grouped_indices.items()}

for traps_x, complaints_y in grouped_array.items():
    inner_prob = az.hdi(complaints_y, hdi_prob=0.95)
    outer_prob = az.hdi(complaints_y, hdi_prob=0.99)
    ax.plot(np.full(inner_prob.shape, traps_x), inner_prob, 'k', linewidth=2)
    ax.plot(np.full(outer_prob.shape, traps_x), outer_prob, 'k', linewidth=0.5)
    ax.plot(traps_x, np.mean(complaints_y), 'ko')
ax.set(xlabel='Number of traps', ylabel='Number of complaints')
plt.show()
plt.close()

It looks as if our model finally generates a reasonable posterior predictive distribution for all numbers of traps, and appropriately captures the tails of the data generating process.

## Using our model: Cost forecasts

Our model seems to be fitting well, so now we will go ahead and use the model to help us make a decision about how many traps to put in our buildings. We'll make a forecast for 6 months forward.


In [ ]:
#| output: false

hier_NB_regression_ncp_slopes_mod_mos_predict = '''
functions {
  /*
  * Alternative to neg_binomial_2_log_rng() that 
  * avoids potential numerical problems during warmup
  */
  int neg_binomial_2_log_safe_rng(real eta, real phi) {
    real gamma_rate = gamma_rng(phi, phi / exp(eta));
    if (gamma_rate >= exp(20.79))
      return -9;
      
    return poisson_rng(gamma_rate);
  }
}
data {
  int<lower=1> N;                     
  array[N] int<lower=0> complaints;
  vector<lower=0>[N] traps;
  
  // 'exposure'
  vector[N] log_sq_foot;
  
  // building-level data
  int<lower=1> K;
  int<lower=1> J;
  array[N] int<lower=1, upper=J> building_idx;
  matrix[J,K] building_data;
  
  // month 
  int<lower=1> M; 
  array[N] int<lower=1,upper=M> mo_idx;
  
  // for use in generated quantities
  int<lower=1> M_forward;
  vector[J] log_sq_foot_pred;
}
transformed data {
  // We'll make predictions for 0, 1, 2, ..., 20 traps (can go further too)
  int N_hypo_traps = 21;
  array[N_hypo_traps] int hypo_traps;
  for (i in 1:N_hypo_traps)
    hypo_traps[i] = i - 1;
}
parameters {
  real<lower=0> inv_phi;   // 1/phi (easier to think about prior for 1/phi instead of phi)
  
  vector[J] mu_raw;        // N(0,1) params for non-centered param of building-specific intercepts
  real<lower=0> sigma_mu;  // sd of buildings-specific intercepts
  real alpha;              // 'global' intercept
  vector[K] zeta;          // coefficients on building-level predictors in model for mu
  
  vector[J] kappa_raw;       // N(0,1) params for non-centered param of building-specific slopes
  real<lower=0> sigma_kappa; // sd of buildings-specific slopes
  real beta;                 // 'global' slope on traps variable
  vector[K] gamma;           // coefficients on building-level predictors in model for kappa
  
  vector[M] mo_raw;               // N(0,1) params for non-centered param of AR(1) process
  real<lower=0> sigma_mo;         // sd of month-specific parameters
  real<lower=0,upper=1> rho_raw;  // used to construct AR(1) coefficient
}
transformed parameters {
  real phi = inv(inv_phi);
  
  // non-centered parameterization of building-specific intercepts and slopes
  vector[J] mu = alpha + building_data * zeta + sigma_mu * mu_raw;
  vector[J] kappa = beta + building_data * gamma + sigma_kappa * kappa_raw;
  
  // AR(1) process priors
  real rho = 2.0 * rho_raw - 1.0;
  vector[M] mo = sigma_mo * mo_raw;
  mo[1] /= sqrt(1 - rho^2);
  for (m in 2:M) {
    mo[m] += rho * mo[m-1];
  }
}
model {
  inv_phi ~ normal(0, 1);
  
  kappa_raw ~ normal(0,1) ;
  sigma_kappa ~ normal(0, 1);
  beta ~ normal(-0.25, 1);
  gamma ~ normal(0, 1);
  
  mu_raw ~ normal(0,1) ;
  sigma_mu ~ normal(0, 1);
  alpha ~ normal(log(4), 1);
  zeta ~ normal(0, 1);
  
  mo_raw ~ normal(0,1);
  sigma_mo ~ normal(0, 1);
  rho_raw ~ beta(10, 5);
  

  { 
  // within a local block we can declare new temporary variables  
  vector[N] eta = mu[building_idx] + kappa[building_idx] .* traps + mo[mo_idx] + log_sq_foot;
  complaints ~ neg_binomial_2_log(eta, phi);
  
  /* 
  alternatively we can use a 'target +=' statement, which is equivalent to 
  above except that using '~' drops constants that don't affect inferences:
  
  target += neg_binomial_2_log_lpmf(complaints | eta, phi);
  */
  }
  
}
generated quantities {
  // we'll predict number of complaints and revenue lost for each building
  // at each hypothetical number of traps for M_forward months in the future
  array[J,N_hypo_traps] int y_pred;
  matrix[J,N_hypo_traps] rev_pred;
  
  for (j in 1:J) {
    for (i in 1:N_hypo_traps) {
      array[M_forward] int y_pred_by_month;
      vector[M_forward] mo_forward;
      
      mo_forward[1] = normal_rng(rho * mo[M], sigma_mo);
      for (m in 2:M_forward) 
        mo_forward[m] = normal_rng(rho * mo_forward[m-1], sigma_mo); 
        
      for (m in 1:M_forward) {
        real eta = mu[j] + kappa[j] * hypo_traps[i] + mo_forward[m] + log_sq_foot_pred[j];
        y_pred_by_month[m] = neg_binomial_2_log_safe_rng(eta, phi);
      }
      
      y_pred[j,i] = sum(y_pred_by_month);
      
      // were were told every 10 complaints has additional exterminator cost of $100, 
      // so $10 lose per complaint.
      rev_pred[j,i] = y_pred[j,i] * (-10.0);
    }
  }
}
'''

comp_rev = StanModel('../stan_models/hier_NB_regression_ncp_slopes_mod_mos_predict', hier_NB_regression_ncp_slopes_mod_mos_predict)

An important input to the revenue model is how much revenue is lost due to each complaint. The client has a policy that for every 10 complaints, they'll call an exterminator costing the client \$100, so that'll amount to \$10 per complaint.


In [ ]:
#| output: false
rev_model = comp_rev.sample(
    data=stan_dat_hier,
    chains=4,
    parallel_chains=4,
    adapt_delta=0.9,
    show_progress=False
)

az_samps_rev_model = az.from_cmdstanpy(rev_model, posterior_predictive=['rev_pred', 'y_pred'], observed_data=stan_dat_hier)

Below we've generated our revenue curves for the buildings. These charts will give us precise quantification of our uncertainty around our revenue projections at any number of traps for each building.

A key input to our analysis will be the cost of installing bait stations. We're simulating the number of complaints we receive over the course of a year, so we need to understand the cost associated with maintaining each bait station over the course of a year. There's the cost attributed to the raw bait station, which is the plastic housing and the bait material, a peanut-buttery substance that's injected with insecticide. The cost of maintaining one bait station for a year plus monthly replenishment of the bait material is about \$20.


In [ ]:
N_traps = np.arange(20+1)
costs = N_traps * 10

We'll also need labor for maintaining the bait stations, which need to be serviced every two months. If there are fewer than five traps, our in-house maintenance staff can manage the stations (about one hour of work every two months at \$20/hour), but above five traps we need to hire outside pest control to help out. They're a bit more expensive, so we've put their cost at \$30 /hour. Each five traps should require an extra person-hour of work, so that's factored in as well. The marginal person-person hours above five traps are at the higher pest-control-labor rate.


In [ ]:
N_months_forward = 12
N_months_labor = N_months_forward // 2
hourly_rate_low = 20
hourly_rate_high = 30
costs += (
    ((N_traps < 5) & (N_traps > 0)) * (N_months_labor * hourly_rate_low) +
    ((N_traps >= 5) & (N_traps < 10)) * (N_months_labor * (hourly_rate_low + 1 * hourly_rate_high)) +
    ((N_traps >= 10) & (N_traps < 15)) * (N_months_labor * (hourly_rate_low + 2 * hourly_rate_high)) +
    (N_traps >= 15) * (N_months_labor * (hourly_rate_low + 3 * hourly_rate_high))
)

We can now plot curves with number of traps on the x-axis and profit/loss forecasts and uncertainty intervals on the y-axis.


In [ ]:
# total and mean profit
tot_profit = az_samps_rev_model.posterior_predictive.rev_pred - costs
mean_profit = tot_profit.median(dim=['chain', 'draw'])

# lower and upper ends of 50% central interval
lower_profit = tot_profit.quantile(q=0.25, dim=['chain', 'draw'])
upper_profit = tot_profit.quantile(q=0.75, dim=['chain', 'draw'])

profit_df = pd.DataFrame({
    'profit': mean_profit.to_numpy().flatten(),
    'lower': lower_profit.to_numpy().flatten(),
    'upper': upper_profit.to_numpy().flatten(),
    'traps': np.tile(N_traps, N_buildings),
    'building': np.repeat(np.arange(1, N_buildings+1), np.max(N_traps)+1)
})

In [ ]:
plt.clf()

fig, axes = plt.subplots(5, 2, figsize=(5,11), sharex=True)

for idx, ax in enumerate(axes.flat):
  data = profit_df[profit_df['building'] == idx+1]
  profit_y = data['profit']
  traps_x = data['traps']
  ax.plot(traps_x, profit_y)
  ax.fill_between(traps_x, y1=data['lower'], y2=data['upper'], alpha=0.2)

fig.supxlabel('Traps')
fig.supylabel('Profit')
plt.show()
plt.close()

We can can see that the optimal number of bait stations differs by building.

### Exercise for the reader

-   How would we build a revenue curve for a new building?

-   Let's say our utility function is revenue. If we wanted to maximize expected revenue, we can take expectations at each station count for each building, and choose the trap numbers that maximizes expected revenue. This will be called a maximum revenue strategy. How can we generate the distribution of portfolio revenue (i.e. the sum of revenue across all the buildings) under the maximum revenue strategy from the the draws of `rev_pred` we already have?

## Gaussian process instead of AR(1)

### Joint density for AR(1) process

We can derive the joint distribution for the AR(1) process before we move to the Gaussian process (GP) which will give us a little more insight into what a GP is. Remember that we've specified the AR(1) prior as:

$$
\begin{align*}
\texttt{mo}_1 &\sim \text{Normal}\left(0, \frac{\sigma_\texttt{mo}}{\sqrt{1 - \rho^2}}\right) \\
\texttt{mo}_t &\sim \text{Normal}\left(\rho \, \texttt{mo}_{t-1}, \sigma_\texttt{mo}\right) \forall t > 1
\end{align*}
$$

Rewriting our process in terms of the errors will make the derivation of the joint distribution clearer

$$
\begin{align*}
\texttt{mo}_1 &\sim \text{Normal}\left(0, \frac{\sigma_\texttt{mo}}{\sqrt{1 - \rho^2}}\right) \\
\texttt{mo}_t &= \rho \, \texttt{mo}_{t-1} + \sigma_\texttt{mo}\epsilon_t  \\
\epsilon_t &\sim \text{Normal}\left(0, 1\right)
\end{align*}
$$

Given that our first term $\texttt{mo}_1$ is normally distributed, and subsequent terms are sums of normal random variables, we can see that jointly the vector, `mo`, with the $t$-th element equally the scalar $\texttt{mo}_t$, is multivariate normal, with mean zero (which we derived above). More formally, if we have a vector $x \in \mathbb{R}^M$ which is multivariate normal, $x \sim \text{MultiNormal}(0, \Sigma)$ and we left-multiply $x$ by a nonsingular matrix $L \in \mathbb{R}^{M\times M}$, $y = Lx \sim \text{MultiNormal}(0, L\Sigma L^T)$. We can use this fact to show that our vector `mo` is jointly multivariate normal.

Just as before with the noncentered parameterization, we'll be taking a vector $\texttt{mo_raw} \in \mathbb{R}^M$ in which each element is univariate $\text{Normal}(0,1)$ and transforming it into `mo`, but instead of doing the transformation with scalar transformations like in the section **Time varying effects and structured priors**, we'll do it with linear algebra operations. The trick is that by specifying each element of $\texttt{mo_raw}$ to be distributed $\text{Normal}(0,1)$ we are implicitly defining $\texttt{mo_raw} \sim \text{MultiNormal}(0, I_M)$, where $I_M$ is the identity matrix of dimension $M \times M$. Then we do a linear transformation using a matrix $L$ and assign the result to `mo` like $\texttt{mo} = L\times\texttt{mo_raw}$ so $\texttt{mo} \sim \text{MultiNormal}(0, LI_M L^T)$ and $LI_M L^T = LL^T$.

Consider the case where we have three elements in `mo` and we want to make figure out the form for $L$.

The first element of `mo` is fairly straightforward, because it mirrors our earlier parameterization of the AR(1) prior. The only difference is that we're explicitly adding the last two terms of `mo_raw` into the equation so we can use matrix algebra for our transformation.

$$
\texttt{mo}_1 = \frac{\sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1 + 0 \times \texttt{mo_raw}_2 + 0 \times \texttt{mo_raw}_3\\
$$

The second element is a bit more complicated:

$$
\begin{align*}
\texttt{mo}_2 &= \rho \texttt{mo}_1 + \sigma_{\texttt{mo}}\,\texttt{mo_raw}_2 + 0 \times \texttt{mo_raw}_3  \\
 &= \rho \left(\frac{\sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1\right) + \sigma_{\texttt{mo}}\,\texttt{mo_raw}_2 + 0 \times \texttt{mo_raw}_3  \\[5pt]
 &= \frac{\rho \sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1 + \sigma_{\texttt{mo}}\,\texttt{mo_raw}_2 + 0 \times \texttt{mo_raw}_3  \\
\end{align*}
$$

While the third element will involve all three terms

$$
\begin{align*}
\texttt{mo}_3 &= \rho \, \texttt{mo}_2 + \sigma_{\texttt{mo}}\,\texttt{mo_raw}_3 \\
 &= \rho \left(\frac{\rho \sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1 + \sigma_{\texttt{mo}}\,\texttt{mo_raw}_2\right) + \sigma_{\texttt{mo}} \texttt{mo_raw}_3  \\[5pt]
&= \frac{\rho^2 \sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1 + \rho \, \sigma_{\texttt{mo}}\,\texttt{mo_raw}_2 +  \sigma_{\texttt{mo}}\,\texttt{mo_raw}_3  \\
\end{align*}
$$

Writing this all together:

$$
\begin{align*}
\texttt{mo}_1 &= \frac{\sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1 + 0 \times \texttt{mo_raw}_2 + 0 \times \texttt{mo_raw}_3\\[3pt]
\texttt{mo}_2 &= \frac{\rho \sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1 + \sigma_{\texttt{mo}}\,\texttt{mo_raw}_2 + 0 \times \texttt{mo_raw}_3  \\[3pt]
\texttt{mo}_3 &= \frac{\rho^2 \sigma_{\texttt{mo}}}{\sqrt{1 - \rho^2}} \times \texttt{mo_raw}_1 + \rho \, \sigma_{\texttt{mo}}\,\texttt{mo_raw}_2 +  \sigma_{\texttt{mo}}\,\texttt{mo_raw}_3  \\
\end{align*}
$$

Separating this into a matrix of coefficients $L$ and the vector `mo_raw`:

$$
\texttt{mo} = \begin{bmatrix} \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & 0 & 0 \\
              \rho \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & \sigma_\texttt{mo} & 0 \\
              \rho^2 \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & \rho \,\sigma_\texttt{mo}  & \sigma_\texttt{mo}
              \end{bmatrix} \times \texttt{mo_raw}
$$

If we multiply $L$ on the right by its transpose $L^T$, we'll get expressions for the covariance matrix of our multivariate random vector `mo`:

$$
\begin{bmatrix} \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & 0 & 0 \\
\rho \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & \sigma_\texttt{mo} & 0 \\
\rho^2 \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & \rho \,\sigma_\texttt{mo}  & \sigma_\texttt{mo}
\end{bmatrix} \times
\begin{bmatrix} \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & \rho \sigma_\texttt{mo} / \sqrt{1 - \rho^2} & \rho^2 \sigma_\texttt{mo} / \sqrt{1 - \rho^2} \\
0 & \sigma_\texttt{mo} & \rho \,\sigma_\texttt{mo} \\
0 & 0  & \sigma_\texttt{mo}
\end{bmatrix}
$$

which results in:

$$
\begin{bmatrix} \sigma^2_\texttt{mo} / (1 - \rho^2) & \rho \, \sigma^2_\texttt{mo} / (1 - \rho^2) &  \rho^2 \, \sigma^2_\texttt{mo} / (1 - \rho^2)\\
\rho \, \sigma^2_\texttt{mo} / (1 - \rho^2)  & \sigma^2_\texttt{mo} / (1 - \rho^2)  & \rho \, \sigma^2_\texttt{mo} / (1 - \rho^2) \\
\rho^2 \sigma^2_\texttt{mo} / (1 - \rho^2) & \rho \, \sigma^2_\texttt{mo} / (1 - \rho^2) & \sigma^2_\texttt{mo} / (1 - \rho^2)
\end{bmatrix}
$$

We can simplify this result by dividing the matrix by $\sigma^2_\texttt{mo} / (1 - \rho^2)$ to get

$$
\begin{bmatrix} 1 & \rho  &  \rho^2 \\
\rho  & 1  & \rho  \\
\rho^2 & \rho & 1
\end{bmatrix}
$$

This should generalize to higher dimensions pretty easily. We could replace the `mo` variable and its definition (inside `transformed parameters` block) in Stan code in `hier_NB_regression_ncp_slopes_mod_mos.stan` with the following:

```         
vector[M] mo;
{
  matrix[M,M] A = rep_matrix(0, M, M);
  A[1,1] = sigma_mo / sqrt(1 - rho^2);
  for (m in 2:M)
    A[m,1] = rho^(m-1) * sigma_mo / sqrt(1 - rho^2);
  for (m in 2:M) {
    A[m,m] = sigma_mo;
    for (i in (m + 1):M)
      A[i,m] = rho^(i-m) * sigma_mo;
  }
  mo = A * mo_raw;
}
```

It's important to the note that the existing block of Stan code is doing the exact same calculations but more efficiently.

### Cholesky decomposition

Note that if we only knew the covariance matrix of our process, say a matrix called $\Sigma$, and we had a way of decomposing $\Sigma$ into $L L^T$ we wouldn't need to write out the equation for the vector. Luckily, there is a matrix decomposition called the **Cholesky decomposition** that does just that. The Stan function for the composition is called `cholesky_decompose`. Instead of writing out the explicit equation, we could do the following:

```         
vector[M] mo;
{
  mo = cholesky_decompose(Sigma) * mo_raw;
}
```

provided we've defined `Sigma` appropriately elsewhere in the transformed parameter block. Note that the matrix $L$ is lower-triangular (i.e. all elements in the upper right-hand triangle of the matrix are zero).

We've already derived the covariance matrix `Sigma` for the three-dimensional AR(1) process above by explicitly calculating $L L^T$, but we can do so using the rules of covariance and the way our process is defined. We already know that each element of $\texttt{mo}_t$ has marginal variance $\sigma^2_\texttt{mo} / (1 - \rho^2)$, but we don't know the covariance of $\texttt{mo}_t$ and $\texttt{mo}_{t+h}$. We can do so recursively. First we derive the covariance for two elements of $\texttt{mo}_t$ separated by one month:

$$
\begin{align*}
\text{Cov}(\texttt{mo}_{t+1},\texttt{mo}_{t}) &= \text{Cov}(\rho \, \texttt{mo}_{t} + \sigma_\texttt{mo}\epsilon_{t+1},\texttt{mo}_{t}) \\
\text{Cov}(\texttt{mo}_{t+1},\texttt{mo}_{t}) &= \rho \text{Cov}(\texttt{mo}_{t},\texttt{mo}_{t}) + \sigma_\texttt{mo}\text{Cov}(\epsilon_{t+1},\texttt{mo}_{t}) \\
\text{Cov}(\texttt{mo}_{t+1},\texttt{mo}_{t}) &= \rho \text{Var}(\texttt{mo}_t) + 0
\end{align*}
$$

Then we define the covariance for $\text{Cov}(\texttt{mo}_{t+h},\texttt{mo}_{t})$ in terms of $\text{Cov}(\texttt{mo}_{t+h-1},\texttt{mo}_{t})$

$$
\begin{align*}
\text{Cov}(\texttt{mo}_{t+h},\texttt{mo}_{t}) &= \text{Cov}(\rho \, \texttt{mo}_{t+h-1} + \sigma_\texttt{mo}\epsilon_{t+h},\texttt{mo}_{t}) \\
\text{Cov}(\texttt{mo}_{t+h},\texttt{mo}_{t}) &= \rho \, \text{Cov}(\texttt{mo}_{t+h-1},\texttt{mo}_{t}) \\
\end{align*}
$$

Which we can use to recursively get at the covariance we need:

$$
\begin{align*}
\text{Cov}(\texttt{mo}_{t+h},\texttt{mo}_{t}) &= \rho \, \text{Cov}(\texttt{mo}_{t+h-1},\texttt{mo}_{t}) \\
\text{Cov}(\texttt{mo}_{t+h},\texttt{mo}_{t}) &= \rho \,( \rho \, \text{Cov}(\texttt{mo}_{t+h-2},\texttt{mo}_{t}) )\\
\dots \\
\text{Cov}(\texttt{mo}_{t+h},\texttt{mo}_{t}) &= \rho^h \, \text{Cov}(\texttt{mo}_{t},\texttt{mo}_{t}) \\
\text{Cov}(\texttt{mo}_{t+h},\texttt{mo}_{t}) &= \rho^h \, \sigma_\texttt{mo}^2/(1 - \rho^2) \\
\end{align*}
$$

Writing this in Stan code to replace the `mo` variable and its definition in `hier_NB_regression_ncp_slopes_mod_mos.stan` we would get:

```         
vector[M] mo;
{
  matrix[M,M] Sigma;
  for (m in 1:M) {
    Sigma[m,m] = 1.0;
    for (i in (m + 1):M) {
      Sigma[i,m] = rho^(i - m);
      Sigma[m,i] = Sigma[i,m];
    }
  }
  Sigma = Sigma * sigma_mo^2 / (1 - rho^2);
  mo = cholesky_decompose(Sigma) * mo_raw;
}
```

### Extension to Gaussian processes

The prior we defined for `mo` is strictly speaking a Gaussian process. It is a stochastic process that is distributed as jointly multivariate normal for any finite value of $M$. Formally, we could write the above prior for `mo` like so:

$$
\begin{align*}
  \sigma_\texttt{mo} &\sim \text{Normal}(0, 1) \\
  \rho &\sim \text{GenBeta}(-1,1,10, 5) \\
  \texttt{mo}_t &\sim \text{GP}\left( 0, K(t | \sigma_\texttt{mo},\rho) \right) \\
\end{align*}
$$

The notation $K(t | \sigma_\texttt{mo},\rho)$ defines the covariance matrix of the process over the domain $t$, which is months.

In other words:

$$
\text{Cov}(\texttt{mo}_t,\texttt{mo}_{t+h}) = k(t, t+h | \sigma_\texttt{mo}, \rho)
$$

We've already derived the covariance for our process. What if we want to use a different definition of `Sigma`?

As the above example shows defining a proper covariance matrix will yield a proper multivariate normal prior on a parameter. We need a way of defining a proper covariance matrix. These are symmetric positive definite matrices. It turns out there is a class of functions that define proper covariance matrices, called **kernel functions**. These functions are applied elementwise to construct a covariance matrix, $K$:

$$
K_{[t,t+h]} = k(t, t+h | \theta)
$$

where $\theta$ are the hyperparameters that define the behavior of covariance matrix.

One such function is called the **exponentiated quadratic function**, and it happens to be implemented in Stan as `gp_exp_quad_cov`. The function is defined as:

$$
\begin{align*}
  k(t, t+h | \theta) &= \alpha^2  \exp \left( - \dfrac{1}{2\ell^2} ((t+h) - t)^2 \right) \\
  &= \alpha^2  \exp \left( - \dfrac{h^2}{2\ell^2} \right)
\end{align*}
$$

The exponentiated quadratic kernel has two components to theta, $\alpha$, the marginal standard deviation of the stochastic process $f$ and $\ell$, the process length-scale.

The length-scale defines how quickly the covariance decays between time points, with large values of $\ell$ yielding a covariance that decays slowly, and with small values of $\ell$ yielding a covariance that decays rapidly. It can be seen interpreted as a measure of how nonlinear the `mo` process is in time.

The marginal standard deviation defines how large the fluctuations are on the output side, which in our case is the number of roach complaints per month across all buildings. It can be seen as a scale parameter akin to the scale parameter for our building-level hierarchical intercept, though it now defines the scale of the monthly deviations.

This kernel's defining quality is its smoothness; the function is infinitely differentiable. That will present problems for our example, but if we add some noise the diagonal of our covariance matrix, the model will fit well.

$$
k(t, t+h | \theta) = \alpha^2  \exp \left( - \dfrac{h^2}{2\ell^2} \right) + \text{if } h = 0, \, \sigma^2_\texttt{noise} \text{ else } 0
$$

### Compiling the GP model


In [ ]:
hier_NB_regression_ncp_slopes_mod_mos_gp = '''
functions {
  int neg_binomial_2_log_safe_rng(real eta, real phi) {
    real phi_div_exp_eta;
    real gamma_rate;
    phi_div_exp_eta = phi/exp(eta);
    gamma_rate = gamma_rng(phi, phi_div_exp_eta);
    if (gamma_rate >= exp(20.79))
      return -9;
    return poisson_rng(gamma_rate);
  }
}
data {
  int<lower=1> N;
  int<lower=1> M;
  int<lower=1> K;
  array[N] int complaints;
  vector[N] traps;
  int<lower=1> J;
  array[N] int<lower=1, upper=J> building_idx;
  matrix[J,K] building_data;
  vector[N] log_sq_foot;
  array[N] int<lower=1> mo_idx;
}
transformed data {
  array[M] real mo_gp_vec;
  for (m in 1:M)
    mo_gp_vec[m] = m;
}
parameters {
  real alpha;
  real<lower=0> sigma_mu;
  real<lower=0> sigma_kappa;
  vector[J] mu_raw;
  vector[J] kappa_raw;
  real beta;
  real<lower=0> inv_phi;
  vector[K] zeta;
  vector[K] gamma;
  // GP prior parameters
  vector[M] gp_raw;
  real<lower=0> gp_len;
  real<lower=0> sigma_gp;
  real<lower=0> sigma_noise;
  vector[M] mo_noise_raw;
}
transformed parameters {
  vector[J] mu = alpha + building_data * zeta + sigma_mu * mu_raw;
  vector[J] kappa = beta + building_data * gamma + sigma_kappa * kappa_raw;
  vector[M] mo_noise = sigma_noise * mo_noise_raw;
  real phi = inv(inv_phi);
  vector[M] gp_exp_quad;
  vector[M] gp;
  {
    matrix[M, M] C = gp_exp_quad_cov(mo_gp_vec, sigma_gp, gp_len);
    real var_noise = square(sigma_noise);
    matrix[M, M] L_C;
    for (m in 1:M)
      C[m,m] += 1e-12;
    L_C = cholesky_decompose(C);
    gp_exp_quad = L_C * gp_raw;
  }
  
  // gp is sum of monthly noise and the smoothly varying process
  gp = mo_noise + gp_exp_quad;
}
model {
  beta ~ normal(-0.25, 1);
  mu_raw ~ normal(0,1);
  kappa_raw ~ normal(0,1);
  sigma_mu ~ normal(0, 1);
  sigma_kappa ~ normal(0, 1);
  alpha ~ normal(log(4), 1);
  zeta ~ normal(0, 1);
  gamma ~ normal(0, 1);
  inv_phi ~ normal(0, 1);
  
  // GP priors
  gp_raw ~ normal(0, 1);
  gp_len ~ gamma(10, 2);
  sigma_gp ~ normal(0, 1);
  
  sigma_noise ~ normal(0, 1);
  mo_noise_raw ~ normal(0, 1);
  
  complaints ~ neg_binomial_2_log(mu[building_idx] + kappa[building_idx] .* traps 
                                 + gp[mo_idx] + log_sq_foot, phi);
} 
generated quantities {
  array[N] int y_rep;

  for (n in 1:N) 
    y_rep[n] = neg_binomial_2_log_safe_rng(mu[building_idx[n]] + kappa[building_idx[n]] * traps[n]
                                          + gp[mo_idx[n]] + log_sq_foot[n],
                                          phi);
}
'''

comp_model_NB_hier_gp = StanModel('../stan_models/hier_NB_regression_ncp_slopes_mod_mos_gp', hier_NB_regression_ncp_slopes_mod_mos_gp)

### Fitting the GP model to data


In [ ]:
fitted_model_NB_hier_gp = comp_model_NB_hier_gp.sample(
    data=stan_dat_hier,
    chains=4,
    parallel_chains=4,
    adapt_delta=0.9,
    show_progress=False
)

samps_gp = az.from_cmdstanpy(fitted_model_NB_hier_gp, posterior_predictive='y_rep', observed_data=stan_dat_hier)

### Examining the fit

Let's look at the prior vs. posterior for the GP length scale parameter:


In [ ]:
plt.clf()
sns.kdeplot(samps_gp.posterior.gp_len.to_numpy().flatten(), label='Posterior')
sns.kdeplot(np.random.gamma(10, 1/2, 4000), label='Prior')
plt.legend()
plt.show()
plt.close()

From the plot above it only looks like we learned a small amount, however we can see a bigger difference between the prior and posterior if we consider how much we learned about the ratio of `sigma_gp` to the length scale `gp_len`:


In [ ]:
plt.clf()
sns.kdeplot(samps_gp.posterior.sigma_gp.to_numpy().flatten() / samps_gp.posterior.gp_len.to_numpy().flatten(), label='Posterior')
sns.kdeplot(np.abs(np.random.normal(size=4000)) / np.random.gamma(10, 1/2, 4000), label='Prior')
plt.legend()
plt.show()
plt.close()

This is a classic problem with Gaussian processes. Marginally, the length-scale parameter isn't very well-identified by the data, but jointly the length-scale and the marginal standard deviation are well-identified.

And let's compare the estimates for the time varying parameters between the AR(1) and GP. In this case the posterior mean of the time trend is essentially the same for the AR(1) and GP priors but the 50% uncertainty intervals are narrower for the AR(1):


In [ ]:
from matplotlib.lines import Line2D 

plt.clf()
fig, ax = plt.subplots()

az.plot_hdi(np.arange(1,36+1), 
            hdi_data=az.hdi(samps_gp.posterior.gp, hdi_prob=0.5), 
            smooth=False, 
            color='C1',
            ax=ax)
az.plot_hdi(np.arange(1,36+1), 
            hdi_data=az.hdi(az_samps_NB_hier_mos.posterior.mo, hdi_prob=0.5), 
            smooth=False, 
            color='C0',
            ax=ax)

legend_handles = [
    Line2D([0], [0], color='C1', lw=2, label="GP"),
    Line2D([0], [0], color='C0', lw=2, label="AR1")
]

ax.legend(handles=legend_handles, loc='lower right')
ax.set(xlabel='Time', ylabel='m')
plt.show()
plt.close()

The way we coded the GP also lets us plot a decomposition of the GP into a monthly noise component (`mo_noise` in the Stan code) and the underlying smoothly varying trend (`gp_exp_quad` in the Stan code):


In [ ]:
plt.clf()
fig, ax = plt.subplots()

time = np.arange(1,36+1)
az.plot_hdi(time, 
            hdi_data=az.hdi(samps_gp.posterior.gp_exp_quad, hdi_prob=0.5), 
            smooth=False, 
            color='C1',
            ax=ax)
az.plot_hdi(time, 
            hdi_data=az.hdi(samps_gp.posterior.mo_noise, hdi_prob=0.5), 
            smooth=False, 
            color='C0',
            ax=ax)

ax.plot(time, samps_gp.posterior.mo_noise.mean(dim=['chain', 'draw']))
ax.plot(time, samps_gp.posterior.gp_exp_quad.mean(dim=['chain', 'draw']))

legend_handles = [
    Line2D([0], [0], color='C1', lw=2, label="Smooth Trend"),
    Line2D([0], [0], color='C0', lw=2, label="Monthly Noise")
]

ax.legend(handles=legend_handles, loc='lower right')
ax.set(xlabel='Time', ylabel='m')
plt.show()
plt.close()